Please run those two cells before running the Notebook!

As those plotting settings are standard throughout the book, we do not show them in the book every time we plot something.

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = "retina"

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

# feel free to modify, for example, change the context to "notebook"
sns.set_theme(context="talk", style="whitegrid", 
              palette="colorblind", color_codes=True, 
              rc={"figure.figsize": [12, 8]})

# Chapter 6 - Time Series Analysis and Forecasting


시계열 데이터는 산업과 연구에서 어디에나 존재합니다. 상업, 기술, 헬스케어, 에너지, 금융 등에서 시계열 데이터의 예를 쉽게 찾아볼 수 있습니다. 우리는 주로 마지막 분야에 관심을 두고 있는데, 그 이유는 거래와 많은 금융/경제 지표에서 시간 차원이 본질적이기 때문입니다. 하지만 거의 모든 비즈니스는 시계열 데이터를 생성하며, 예를 들어 시간이 지남에 따라 수집된 수익 또는 측정된 KPI와 같은 데이터를 다룹니다. 따라서 다음 두 챕터에서 다룰 기술들은 여러분이 업무에서 마주할 수 있는 모든 시계열 분석 과제에 활용될 수 있습니다.

시계열 모델링이나 예측은 종종 다양한 관점에서 접근할 수 있습니다. 가장 인기 있는 두 가지 방법은 통계적 방법과 머신러닝 접근법입니다. 또한, 15장 '금융에서의 딥러닝'에서 시계열 예측에 딥러닝을 사용하는 몇 가지 예시도 다룰 예정입니다. 과거에는 방대한 컴퓨팅 파워가 없었고, 시계열 데이터가 지금처럼 세밀하지 않았을 때(데이터가 어디에서나 그리고 항상 수집되지 않았던 시기), 통계적 접근법이 이 분야를 지배했습니다. 최근에는 상황이 변했고, 시계열 모델이 실제 운영되는 환경에서는 머신러닝 기반 접근법이 선도하고 있습니다. 그렇다고 해서 고전적인 통계적 접근법이 더 이상 유효하지 않다는 뜻은 아닙니다. 실제로, 아주 적은 훈련 데이터(예: 3년간의 월별 데이터)가 있는 경우에는 머신러닝 모델이 패턴을 학습하지 못하는 반면, 통계적 방법이 여전히 최첨단 결과를 제공할 수 있습니다. 또한, 통계적 방법이 최근의 M-경쟁(Spyros Makridakis가 시작한 최대 규모의 시계열 예측 대회)에서 여러 번 우승하는 것을 볼 수 있습니다.

이 챕터에서는 시계열 모델링의 기초를 소개합니다. 먼저 시계열의 구성 요소와 이를 분해 방법을 통해 분리하는 방법을 설명합니다. 이후에는 정상성(stationarity)의 개념을 다루며, 왜 중요한지, 이를 테스트하는 방법, 그리고 원래의 시계열이 정상적이지 않을 경우 정상성을 달성하는 방법에 대해 설명합니다. 그 후, 시계열 모델링에 가장 널리 사용되는 두 가지 통계적 접근법인 지수 평활법과 ARIMA 클래스 모델을 살펴봅니다. 두 경우 모두, 모델을 어떻게 적합시키고 적합도를 평가하며, 시계열의 미래 값을 예측하는 방법을 보여드립니다.

이 챕터에서 다룰 레시피는 다음과 같습니다:
- 시계열 분해
- 시계열의 정상성 테스트
- 시계열의 정상성 수정
- 지수 평활법을 이용한 시계열 모델링
- ARIMA 클래스 모델을 이용한 시계열 모델링
- auto-ARIMA를 이용한 최적의 ARIMA 모델 찾기

## 6.1 Time series decomposition

시계열 분해의 목표 중 하나는 시계열 데이터를 여러 구성 요소로 분해하여 데이터에 대한 이해를 높이는 것입니다. 이를 통해 모델링 복잡성을 파악하고 각 구성 요소를 정확하게 포착 및 모델링하기 위한 접근 방식을 제공합니다.

**시계열 구성 요소**는 체계적 요소(systematic)와 비체계적 요소(non-systematic)로 나뉩니다. 체계적 요소는 일관성이 있으며 모델링할 수 있는 반면, 비체계적 요소는 직접적으로 모델링할 수 없습니다.

- 체계적 요소:
    - **레벨(Level)**: 시계열의 평균 값
    - **추세(Trend)**: 각 시점 사이의 값 변화 추정치, 시계열의 전반적인 방향 (증가/감소)
    - **계절성(Seasonality)**: 고정된 주기적인 패턴에 의해 평균에서 벗어난 변화


- 비체계적 요소:
    - **잡음(Noise)**: 시계열에서 다른 구성 요소를 제거한 후 남은 랜덤한 변동


- 시계열 분해 모델
    - 시계열 분해의 고전적인 접근 방식은 **가법 모델(Additive)** 또는 **승법 모델(Multiplicative)** 중 하나를 사용합니다.


- 가법 모델의 특징:
    - 모델 형태:  
      $
      y(t) = level + trend + seasonality + noise
      $
    - 선형 모델: 시간이 지남에 따라 변화가 일정함
    - 추세는 직선 (선형)
    - 주기성이 일정한 주기와 진폭을 가짐


- 승법 모델의 특징:
    - 모델 형태:  
      $
      y(t) = level \times trend \times seasonality \times noise
      $
    - 비선형 모델: 시간이 지남에 따라 변화가 일정하지 않음 (예: 지수 함수)
    - 비선형적인 추세와 시간이 지남에 따라 주기와 진폭이 변하는 계절성


좀 더 흥미롭게 만들기 위해, 가법적 특성과 승법적 특성이 결합된 시계열을 찾을 수 있습니다. 예를 들어, 가법적인 추세와 승법적인 계절성을 가진 시계열을 생각해볼 수 있습니다. 가능한 조합을 시각화한 아래 그림을 참조하시기 바랍니다. 

<img src="./image/fig_06_01.png" width=800>

그리고 실제 문제는 이처럼 간단하지 않더라도 (잡음이 있는 데이터와 다양한 패턴), 이러한 추상적인 모델은 시계열을 모델링/예측하기 전에 분석할 수 있는 간단한 프레임워크를 제공합니다.    

승법 모델을 사용하고 싶지 않거나 (모델의 가정 때문에) 사용할 수 없는 경우가 있을 수 있습니다. 가능한 해결책 중 하나는 로그 변환을 사용하여 승법 모델을 가법 모델로 변환하는 것입니다:

$
\log(\text{{time}} * \text{{seasonality}} * \text{{residual}}) = \log(\text{{time}}) + \log(\text{{seasonality}}) + \log(\text{{residual}})
$

이 레시피에서는 나스닥 데이터 링크에서 다운로드한 미국 월별 실업률 데이터를 사용하여 시계열 분해를 수행하는 방법을 소개할 것입니다.

### How to do it...

1. Import the libraries and authenticate:

In [3]:
import pandas as pd
import nasdaqdatalink
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
import os

nasdaqdatalink.ApiConfig.api_key = os.environ['NASDAQ_API_KEY']

2. Download the monthly US unemployment rate from years 2010-2019:

In [4]:
df = (
    nasdaqdatalink.get(dataset="FRED/UNRATENSA", 
                       start_date="2010-01-01", 
                       end_date="2019-12-31")
    .rename(columns={"Value": "unemp_rate"})
)
df.head()

DataLinkError: (Status 410) Something went wrong. Please try again. If you continue to have problems, please contact us at connect@data.nasdaq.com.

In [ ]:
# a quick look at the plot
temp_df = df.copy()

temp_df["year"] = temp_df.index.year
temp_df["month"] = temp_df.index.strftime("%b")

sns.lineplot(data=temp_df, 
             x="month", 
             y="unemp_rate", 
             hue="year",
             style="year", 
             legend="full",
             palette="colorblind")

plt.title("Unemployment rate - Seasonal plot")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2);

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_6_2", dpi=200)

Figure 6.2: Seasonal plot of the US unemployment rate in the years 2010 to 2019

그림 6.2에서 우리는 시계열에서 분명한 계절 패턴을 확인할 수 있습니다. COVID-19 팬데믹이 실업률 시계열에서 관찰 가능한 패턴에 급격한 변화를 일으켰기 때문에 이번 분석에는 최신 데이터를 포함하지 않았습니다. 그래프를 생성하는 데 사용된 코드는 3장 '금융 시계열 시각화'에서 사용된 코드와 매우 유사하기 때문에 별도로 표시하지 않았습니다.

3. Add rolling mean and standard deviation:

2단계에서 데이터를 다운로드한 후, 우리는 pandas DataFrame의 rolling 메서드를 사용하여 이동 통계치를 계산했습니다. 월별 데이터를 다루고 있기 때문에 창 크기를 12개월로 지정했습니다. 

In [ ]:
WINDOW_SIZE = 12
df["rolling_mean"] = df["unemp_rate"].rolling(window=WINDOW_SIZE).mean()
df["rolling_std"] = df["unemp_rate"].rolling(window=WINDOW_SIZE).std()
df.plot(title="Unemployment rate")

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_6_3", dpi=200)

Figure 6.3: The US unemployment rate together with the rolling average and standard deviation

4. Carry out seasonal decomposition using the additive model:

우리는 statsmodels 라이브러리의 seasonal_decompose 함수를 사용하여 고전적인 분해를 수행했습니다. 이때, 사용할 모델 유형을 지정했으며, 가능한 값은 가법(additive)과 승법(multiplicative)입니다. seasonal_decompose를 숫자 배열과 함께 사용할 때는 pandas Series 객체를 사용하는 경우를 제외하고 관측값의 주기(freq 인자)를 지정해야 합니다. 결측값이 있거나 시계열의 처음과 끝 부분에 대한 잔차를 보간하고 싶을 경우, 추가 인자인 extrapolate_trend='freq'를 전달할 수 있습니다.

In [ ]:
decomposition_results = seasonal_decompose(df["unemp_rate"], 
                                           model="additive")
# decomposition_results = seasonal_decompose(df["unemp_rate"], 
#                                            model="additive", extrapolate_trend='freq')
(
    decomposition_results
    .plot()
    .suptitle("Additive Decomposition")
)

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_6_4", dpi=200)

Figure 6.4: The seasonal decomposition of the US unemployment rate (using an additive model) 

분해 그래프에서 추세, 계절성, 그리고 랜덤(잔차) 요소로 분리된 시계열을 볼 수 있습니다. 분해가 타당한지 평가하기 위해서는 랜덤 요소를 살펴볼 수 있습니다. 만약 뚜렷한 패턴이 없고(즉, 랜덤 요소가 정말로 무작위이며 시간이 지나도 일관되게 행동한다면), 적합이 타당하다는 의미입니다. 이번 경우에는 잔차의 분산이 데이터셋의 첫 번째 절반에서 약간 더 높은 것으로 보입니다. 이는 일정한 계절 패턴만으로는 분석된 시계열의 계절성을 정확하게 포착하기에 충분하지 않다는 것을 나타낼 수 있습니다.

### There's more

이번 레시피에서 사용한 계절 분해는 가장 기본적인 접근 방식입니다. 이 방식에는 몇 가지 단점이 있습니다:

- 알고리즘이 중심 이동 평균을 사용하여 추세를 추정하기 때문에, 분해를 수행하면 시계열의 처음과 끝 부분에서 추세선(및 잔차)의 값이 누락됩니다.
- 이 접근법으로 추정된 계절 패턴은 매년 반복된다고 가정합니다. 이는 특히 더 긴 시계열에 대해 매우 강한 가정입니다.
- 추세선은 데이터를 지나치게 평활화(smoothing)하는 경향이 있으며, 그 결과 급격하거나 갑작스러운 변동에 적절히 반응하지 못합니다.
- 이 방법은 데이터의 잠재적인 이상치에 대해 강건하지 않습니다.

시간이 지나면서 시계열 분해에 대한 몇 가지 대안적인 접근법이 도입되었습니다. 이 섹션에서는 statsmodels 라이브러리에 구현된 LOESS(STL 분해)를 사용한 계절 및 추세 분해도 다룰 것입니다.

**LOESS**는 **국소 추정 산점도 평활화**(locally estimated scatterplot smoothing)를 의미하며, 비선형 관계를 추정하는 방법입니다.

STL 분해가 어떻게 작동하는지에 대한 세부 사항은 다루지 않겠지만, 다른 접근법에 비해 가지는 장점에 대해 알아두는 것이 좋습니다:

- STL은 어떤 형태의 계절성도 처리할 수 있습니다(다른 방법들처럼 월별 또는 분기별로 제한되지 않음).
- 사용자가 추세의 평활도를 제어할 수 있습니다.
- 계절 구성 요소가 시간에 따라 변화할 수 있으며, 변화율은 사용자가 제어할 수 있습니다.
- STL은 이상치에 더 강건합니다. 추세와 계절 구성 요소의 추정치는 이상치의 영향을 받지 않으며, 이상치는 잔여 구성 요소에서 그 영향이 여전히 드러납니다.

물론, 이 방법이 만능 해결책은 아니며 몇 가지 단점도 있습니다. 예를 들어, STL은 가법 분해에서만 사용할 수 있으며, 거래일이나 달력 변동을 자동으로 고려하지 않습니다.

최근에는 여러 계절성을 처리할 수 있는 STL 분해의 변형이 등장했습니다. 예를 들어, 시간 단위의 데이터 시계열은 일간/주간/월간 계절성을 나타낼 수 있습니다. 이 방법은 **LOESS를 사용한 다중 계절-추세 분해(MSTL)** 라고 하며, 관련 내용은 'See also' 섹션에서 참고할 수 있습니다.

- 추가
    - LOESS는 단순한 국소 평활화 방법으로, 데이터를 특정한 패턴 없이 국소적으로 부드럽게 만드는 기술입니다.
    - STL 분해는 LOESS를 이용하여 시계열 데이터를 추세와 계절성으로 분해하는 보다 구조화된 방법입니다. STL은 LOESS의 평활화 기법을 활용하여 시계열 데이터의 주요 구성 요소를 분리합니다.

Use the STL decomposition:

In [ ]:
from statsmodels.tsa.seasonal import STL

stl_decomposition = STL(df[["unemp_rate"]]).fit()
stl_decomposition.plot() \
                 .suptitle("STL Decomposition")

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_6_5", dpi=200)

Figure 6.5: The STL decomposition of the US unemployment time series 

STL 분해와 고전적인 분해의 그래프가 매우 유사하다는 것을 알 수 있습니다. 그러나 그림 6.5에서는 STL 분해가 고전적인 분해에 비해 가지는 몇 가지 장점과 관련된 미묘한 차이점들이 있습니다. 

- 첫째, 추세 추정치에 누락된 값이 없습니다. 
- 둘째, 계절 구성 요소가 시간이 지남에 따라 천천히 변화하고 있습니다. 

예를 들어, 여러 해 동안 1월의 값을 보면 이를 명확하게 확인할 수 있습니다. STL에서 계절 인자의 기본값은 7로 설정되어 있지만, 이 방법의 제안자들은 더 큰 값을 사용할 것을 권장합니다(7 이상이고 홀수여야 함). 내부적으로, 이 파라미터 값은 계절 구성 요소의 각 값을 추정하는 데 사용할 연속적인 해의 수를 나타냅니다. 더 큰 값을 선택할수록 계절 구성 요소는 더 평활해지며, 이는 시계열에서 관찰되는 변동 중 일부가 계절 구성 요소에 귀속되는 빈도가 줄어듭니다. 추세 인자에 대한 해석도 유사하지만, 이는 추세 구성 요소를 추정하는 데 사용할 연속적인 관측값의 수를 나타냅니다.

또한, STL 분해의 장점 중 하나는 이상치에 대한 강건성이 더 높다는 점을 언급한 바 있습니다. 우리는 robust 인자를 사용하여 데이터에 의존하는 가중치 함수를 활성화할 수 있습니다. 이 경우 LOESS 추정 시 관측값의 가중치를 다시 계산하여, LOWESS(국소 가중 산점도 평활화)가 됩니다. 강건한 추정을 사용할 때, 모델은 잔차 구성 요소의 그래프에서 볼 수 있는 더 큰 오류를 허용할 수 있습니다. 그림 6.6에서는 강건한 추정이 포함된 STL 분해와 포함되지 않은 STL 분해를 사용해 미국 실업률 데이터를 적합시킨 결과를 비교한 내용을 볼 수 있습니다. 이 그림을 생성하는 데 사용된 코드는 이 책의 GitHub 저장소에 있는 노트북을 참조하시기 바랍니다.

Compare the decompositions with and without the robust setting:

In [ ]:
def add_second_stl_to_plot(fig, fitted_stl, labels):
    """
    A helper function adding the 3 components from the second STL fit to the 
    first STL plot
    """
    axs = fig.get_axes()
    comps = ["trend", "seasonal", "resid"]
    for ax, comp in zip(axs[1:], comps):
        series = getattr(fitted_stl, comp)
        if comp == "resid":
            ax.plot(series, marker="o", linestyle="none")
        else:
            ax.plot(series)
            if comp == "trend":
                ax.legend(labels, frameon=False)


stl_robust = STL(df[["unemp_rate"]], robust=True).fit()
stl_non_robust = STL(df[["unemp_rate"]], robust=False).fit()
fig = stl_robust.plot()
add_second_stl_to_plot(fig, stl_non_robust, ["Robust", "Non-robust"])

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_6_6", dpi=200)


Figure 6.6: The effect of using robust estimation in the STL decomposition process 

강건한 추정을 사용했을 때의 효과를 분명히 관찰할 수 있습니다. 더 큰 오류가 허용되며, 분석된 시계열의 처음 몇 년 동안 계절 구성 요소의 모양이 달라집니다. 이 경우 강건한 방식이 더 나은지 비강건한 방식이 더 나은지에 대한 명확한 답은 없습니다. 이는 우리가 분해를 어떤 목적으로 사용하는지에 달려 있습니다. 이 레시피에서 제시한 계절 분해 방법은 간단한 이상치 탐지 알고리즘으로도 사용할 수 있습니다. 예를 들어, 시계열을 분해한 후 잔차를 추출하여 잔차가 3배의 사분위 범위(IQR)를 벗어나는 관측값을 이상치로 표시할 수 있습니다. **kats** 라이브러리는 **OutlierDetector** 클래스에서 이러한 알고리즘을 구현하고 있습니다.


- 계절 분해에 대한 다른 접근법으로는 다음이 포함됩니다:
    - **ARIMA 시계열의 계절 추출(SEATS) 분해**.
    - **X11 분해**—이 분해의 변형은 모든 관측값에 대해 추세-주기(trend-cycle) 구성 요소를 생성하며, 계절 구성 요소가 시간이 지남에 따라 천천히 변할 수 있도록 허용합니다.
    - **Hodrick-Prescott 필터**—이 방법은 계절 분해 접근법이라기보다는 데이터 평활화 기법입니다. 이 방법은 경기 순환과 관련된 단기 변동을 제거하여 장기 추세를 드러냅니다. HP 필터는 거시경제에서 흔히 사용되며, **statsmodels**의 **hpfilter** 함수에서 구현을 찾을 수 있습니다.



Apply the Hodrick-Prescott filter to the unemployment time series:

In [ ]:
from statsmodels.tsa.filters.hp_filter import hpfilter

hp_df = df[["unemp_rate"]].copy()
hp_df["cycle"], hp_df["trend"] = hpfilter(hp_df["unemp_rate"], 129600)
hp_df.plot(subplots=True, title="Hodrick-Prescott filter")

sns.despine()
plt.tight_layout()

### See also 

1. **Bandara, K., Hyndman, R. J., & Bergmeir, C. 2021.**  
   **"MSTL: 다중 계절 패턴을 가진 시계열을 위한 계절-추세 분해 알고리즘"**  
   arXiv preprint arXiv:2107.13462.

2. **Cleveland, R. B., Cleveland, W. S., McRae, J. E., & Terpenning, I. J. 1990.**  
   **"LOESS 기반 계절 추세 분해 절차,"**  
   Journal of Official Statistics 6(1): 3–73.

3. **Hyndman, R.J. & Athanasopoulos, G. 2021.**  
   **"Forecasting: Principles and Practice," 3판, OTexts: 멜버른, 호주.**  
   OTexts.com/fpp3

4. **Sutcliffe, A. 1993.**  
   **"X11 시계열 분해와 샘플링 오류,"**  
   호주 통계청.

## 6.2 Testing for stationarity in time series

시계열 분석에서 가장 중요한 개념 중 하나는 **정상성(stationarity)** 입니다. 쉽게 말하면, **정상적인 시계열** 이란 시간에 따라 그 특성이 변하지 않는 시계열을 말합니다. 즉, 정상성은 데이터 생성 과정의 통계적 속성이 시간이 지나도 변하지 않음을 의미합니다. 정상성을 가정한 시계열에서는 **추세**나 **계절성** 을 볼 수 없어야 합니다. 만약 존재한다면 이는 정상성 가정을 위반하는 것입니다. 반면, **백색 잡음(white noise)** 는 언제 관측하더라도 거의 동일한 특성을 보이기 때문에 정상적입니다.

**추세와 계절성이 없지만 주기적 행동을 보이는 시계열** 도 여전히 정상적일 수 있습니다. 이는 주기가 고정된 것이 아니기 때문에, 주기적인 패턴이 항상 일정하지 않기 때문입니다.

정상성에 대한 보다 공식적인 정의는 여러 가지가 있지만, 여기서는 **약한 정상성(weak stationarity)** 또는 **공분산 정상성(covariance stationarity)** 개념을 사용합니다. 시계열이 공분산 정상성으로 분류되기 위해서는 다음 세 가지 조건을 만족해야 합니다:
1. **시계열의 평균이 일정**해야 합니다.
2. **분산이 유한하고 일정**해야 합니다.
3. **같은 간격에 해당하는 구간 간의 공분산이 일정**해야 합니다.


- 정상성의 중요성
    - **정상성**은 시계열 분석에서 매우 중요한 특성입니다. 왜냐하면 정상적인 시계열은 **미래 예측**이 더 쉬워지기 때문입니다. 시계열이 정상적이면 과거와 미래의 통계적 특성이 동일하므로, 예측 모델을 더 신뢰할 수 있습니다.


- 비정상적 시계열의 단점
    - 1. **분산이 잘못 모델링될 수 있음**.
    - 2. **모델 적합도가 나빠져서 예측 정확도가 떨어짐**.
    - 3. **시간에 의존하는 패턴을 활용할 수 없음**.


- 정상성이 모든 모델에 적용되는 것은 아님
    - **정상성**은 시계열에서 중요한 특성이지만, 모든 통계 모델에 적용되는 것은 아닙니다. 예를 들어, **AR(자기회귀 모델)**, **ARMA**, **ARIMA** 등과 같은 모델에서는 정상성이 중요합니다. 하지만 **시계열 분해**나 **평활화 기법**, 또는 **Facebook의 Prophet**처럼 정상성이 필요하지 않은 모델들도 있습니다.


- 이 레시피에서 소개하는 정상성 테스트 방법:
    - 1. **Augmented Dickey-Fuller (ADF) 테스트**.
    - 2. **Kwiatkowski-Phillips-Schmidt-Shin (KPSS) 테스트**.
    - 3. **(부분) 자기상관함수(PACF/ACF) 플롯**.


이 테스트들은 2010년부터 2019년까지의 **월별 실업률** 데이터를 사용하여 정상성을 검사하는 데 적용됩니다.

### Getting ready

- 우리는 시계열 분해 레시피에서 사용한 것과 동일한 데이터를 사용할 것입니다. 그림 6.3에서 월별 실업률의 이동 평균과 표준 편차를 나타내는 그래프에서 이미 시간이 지남에 따라 하락하는 음의 추세를 확인했으며, 이는 비정상성을 시사합니다.

1. Import the libraries and authenticate:

In [ ]:
import pandas as pd
import nasdaqdatalink

# nasdaqdatalink.ApiConfig.api_key = "YOUR_KEY_HERE"

2. Download the monthly US unemployment rate from years 2010-2019:

In [ ]:
df = (
    nasdaqdatalink.get(dataset="FRED/UNRATENSA", 
                       start_date="2010-01-01", 
                       end_date="2019-12-31")
    .rename(columns={"Value": "unemp_rate"})
)
df.head()

### How to do it...

다음 단계들을 실행하여 미국 월별 실업률 시계열이 정상적인지 테스트합니다.

1. Import the libraries:

In [ ]:
import pandas as pd
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller, kpss

2. Define a function for running the ADF test:

**2단계에서** , 우리는 ADF 테스트를 실행하고 결과를 출력하는 함수를 정의했습니다. **adfuller** 함수를 호출할 때 **autolag="AIC"** 를 지정하여, **Akaike 정보 기준(AIC)** 에 따라 고려할 지연 개수를 자동으로 선택하도록 했습니다. 대안적으로, 지연 개수를 수동으로 선택할 수도 있습니다.

In [ ]:
def adf_test(x):
    """
    정상성을 테스트하기 위한 Augmented Dickey-Fuller 테스트 함수
    
    귀무가설: 시계열이 정상성이 아니다
    대립가설: 시계열이 정상성이다

    매개변수
    ----------
    x : pd.Series / np.array
        정상성 여부를 확인할 시계열 데이터
    
    반환값
    -------
    results: pd.DataFrame
        ADF 테스트 결과가 포함된 DataFrame
    """
    
    indices = ["Test Statistic", "p-value", 
                "# of Lags Used", "# of Observations Used"]
    
    adf_test = adfuller(x, autolag="AIC")
    results = pd.Series(adf_test[0:4], index=indices)
    
    for key, value in adf_test[4].items():
        results[f"Critical Value ({key})"] = value

    return results

In [ ]:
adf_test(df["unemp_rate"])

- ADF 테스트의 귀무가설은 시계열이 정상성이 아니라고 가정합니다. p-value가 0.26인 경우(또는 테스트 통계량이 선택된 신뢰 수준에서 임계값보다 큰 경우), 귀무가설을 기각할 근거가 없으므로, 시계열이 정상성이 아니라고 결론지을 수 있습니다.

3. Define a function for running the KPSS test:

**3단계** 에서, **kpss** 함수에 **regression** 인수를 지정했습니다. **"c"** 값은 시계열이 **수준-정상성(level-stationary)** 이라는 귀무가설에 해당하며, **"ct"** 값은 **추세-정상성(trend-stationary)** 을 나타냅니다(시계열에서 추세를 제거하면 수준-정상성이 된다는 가정). 모든 테스트와 자기상관 그래프에 대해, 우리는 유의 수준을 **5%** 로 설정했으며, 이는 귀무가설(H0)이 참일 때 이를 기각할 확률을 의미합니다.


- **KPSS 테스트**는 시계열이 정상적인지 아닌지를 판단하는 중요한 도구입니다. 여기서 "수준-정상성"과 "추세-정상성"이라는 개념이 다소 어려울 수 있는데, 쉽게 말해:
    - **수준-정상성(level-stationary)**: 시계열 데이터가 평균값(수준)을 중심으로 일정하게 움직이는 상태입니다. 예를 들어, 어떤 경제 지표가 매년 평균적으로 같은 수준을 유지하면서, 큰 변화 없이 작은 변동을 보이는 경우입니다. 여기서 'c' 값은 데이터를 **평균이 일정한 상태**로 보고 분석한다는 의미입니다.  
    - **추세-정상성(trend-stationary)**: 시계열 데이터가 시간이 지나면서 **일정한 추세**를 따라 증가하거나 감소하는데, 이 추세를 제거하면 남은 데이터가 **수준-정상성**을 유지한다는 뜻입니다. 예를 들어, 물가가 매년 조금씩 상승하지만, 그 상승 추세를 제거하면 물가의 작은 변동이 일정하게 남는 상황입니다. 'ct' 값은 데이터를 **추세를 고려한 상태**로 분석한다는 의미입니다.


In [ ]:
def kpss_test(x, h0_type="c"):
    """
    Kwiatkowski-Phillips-Schmidt-Shin (KPSS) 테스트를 사용해 정상성을 확인하는 함수

    귀무가설: 시계열이 정상성이다
    대립가설: 시계열이 정상성이 아니다

    매개변수
    ----------
    x: pd.Series / np.array
        정상성 여부를 확인할 시계열 데이터
    h0_type: str{"c", "ct"}
        KPSS 테스트의 귀무가설을 나타냅니다:
            * "c": 데이터가 상수 주위에서 정상성(기본값)
            * "ct": 데이터가 추세 주위에서 정상성
    
    반환값
    -------
    results: pd.DataFrame
        KPSS 테스트 결과가 포함된 DataFrame
    """
    
    indices = ["Test Statistic", "p-value", "# of Lags"]

    kpss_test = kpss(x, regression=h0_type)
    results = pd.Series(kpss_test[0:3], index=indices)
    
    for key, value in kpss_test[3].items():
        results[f"Critical Value ({key})"] = value

    return results

In [ ]:
kpss_test(df["unemp_rate"])

- KPSS 테스트의 귀무가설은 시계열이 정상성이라고 가정합니다. p-value가 0.01인 경우(또는 테스트 통계량이 선택된 임계값보다 큰 경우), 우리는 대립가설을 지지하며 귀무가설을 기각할 근거가 있습니다. 이는 시계열이 정상성이 아니라는 것을 나타냅니다.

4. Generate the ACF/PACF plots:

In [ ]:
N_LAGS = 40
SIGNIFICANCE_LEVEL = 0.05

fig, ax = plt.subplots(2, 1)
plot_acf(df["unemp_rate"], ax=ax[0], lags=N_LAGS, 
         alpha=SIGNIFICANCE_LEVEL)
plot_pacf(df["unemp_rate"], ax=ax[1], lags=N_LAGS, 
          alpha=SIGNIFICANCE_LEVEL)

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_6_7", dpi=200)

Figure 6.7: Autocorrelation and Partial Autocorrelation plots of the unemployment rate 

ACF 그래프에서 95% 신뢰 구간(선택한 5% 유의 수준에 해당)을 초과하는 유의미한 자기상관이 있음을 확인할 수 있습니다. 또한, PACF 그래프에서는 지연 1과 4에서 유의미한 자기상관이 나타납니다.

### There's more

이번 레시피에서 우리는 **statsmodels** 라이브러리를 사용하여 정상성 테스트를 수행했습니다. 그러나 결과를 깔끔하게 요약하기 위해 커스텀 함수를 만들어야 했습니다. 대안으로, **arch** 라이브러리에서 제공하는 정상성 테스트를 사용할 수 있습니다(이 라이브러리는 **9장: GARCH 클래스 모델을 사용한 변동성 모델링** 에서 더 자세히 다룰 것입니다).

**ADF 테스트**는 다음 코드 조각을 사용하여 수행할 수 있습니다:

1. Carry out the ADF test using the `arch` library:

In [ ]:
from arch.unitroot import ADF
adf = ADF(df["unemp_rate"])
print(adf.summary().as_text())

2. Carry out the Zivot-Andrews test using the `arch` library:

In [ ]:
from arch.unitroot import ZivotAndrews
za = ZivotAndrews(df["unemp_rate"])
print(za.summary().as_text())

In [ ]:
from arch.unitroot import ADF
adf = ADF(df["unemp_rate"])
print(adf.summary().as_text())

이는 관련된 모든 정보를 포함한 깔끔하게 포맷된 출력을 반환합니다:

**arch** 라이브러리는 다음과 같은 추가 정상성 테스트도 포함하고 있습니다:
- **Zivot-Andrews 테스트** (**statsmodels** 에서도 제공)
- **Phillips-Perron (PP) 테스트** (**statsmodels** 에서는 제공되지 않음)

ADF 및 KPSS 테스트의 잠재적인 단점은 **구조적 변화(structural break)**, 즉 시계열의 평균 또는 다른 매개변수에서 갑작스러운 변화를 허용하지 않는다는 점입니다. **Zivot-Andrews 테스트** 는 시계열에서 알 수 없는 시점에 발생하는 **단일 구조적 변화** 의 가능성을 고려할 수 있습니다.

We can run the test using the following snippet:

In [ ]:
from arch.unitroot import ZivotAndrews
za = ZivotAndrews(df["unemp_rate"])
print(za.summary().as_text())

테스트의 **p-value** 에 근거하여, 시계열이 정상성이 아니라는 귀무가설을 기각할 수 없습니다.

### See also

- 추가적인 정상성 테스트에 대한 자세한 내용은 다음을 참조하십시오:  
    - Phillips, P. C. B. & P. Perron, 1988. “Testing for a unit root in time series regression,” Biometrika 75: 335-346.  
    - Zivot, E. & Andrews, D.W.K., 1992. “Further evidence on the great crash, the oil-price shock, and the unit-root hypothesis,” Journal of Business & Economic Studies, 10: 251-270.

## 6.3 Correcting for stationarity in time series

이전 레시피에서는 주어진 시계열이 정상성인지 조사하는 방법을 배웠습니다. 이번 레시피에서는 **정상성이 아닌 시계열** 을 다음과 같은 변환 중 하나(또는 여러 개)를 사용하여 **정상성** 으로 만드는 방법을 살펴보겠습니다:

- **디플레이션**: 소비자 물가지수(CPI)를 사용하여 화폐 시계열에서 **인플레이션을 반영** 하는 방법
- **자연 로그 적용**: 잠재적인 **지수 추세**를 선형에 가깝게 만들고, 시계열의 **분산을 줄이는** 방법
- **차분(Differencing)**: 현재 관측값과 과거의 지연된 관측값(현재 관측값보다 x 시간 단위 이전의 값) 간의 **차이를 계산** 하는 방법

이번 실습에서는 **2000년부터 2010년까지의 월별 금 가격** 을 사용할 것입니다. 이 표본을 선택한 이유는 이 기간 동안 금 가격이 꾸준히 상승하는 **추세** 를 보이기 때문입니다. 따라서 이 시계열은 확실히 **정상성이 아닙니다**.

### How to do it...

다음 단계를 수행하여 시계열을 비정상성에서 정상성으로 변환하십시오:



1. Import the libraries, authenticate and update the inflation data:

In [ ]:
import pandas as pd
import numpy as np
import nasdaqdatalink
import cpi
from datetime import date
from chapter_6_utils import test_autocorrelation
import os

nasdaqdatalink.ApiConfig.api_key = os.environ['NASDAQ_API_KEY']

# update the CPI data (if needed)
# cpi.update()

2. Download the prices of gold for 2000-2010 and resample to monthly values:

라이브러리를 가져오고, 인증하고, CPI 데이터를 업데이트한 후, Nasdaq Data Link에서 월별 금 가격 데이터를 다운로드했습니다. 

시계열에 중복된 값이 일부 있었는데, 예를 들어 2000-04-28과 2000-04-30에 동일한 값이 기록된 경우가 있었습니다. 이를 처리하기 위해 데이터를 월별 빈도로 리샘플링하여 해당 월에 사용 가능한 마지막 값을 사용했습니다. 이렇게 함으로써 각 월의 잠재적인 중복 값만 제거하고 실제 값을 변경하지 않았습니다.

In [ ]:
df = (
    nasdaqdatalink.get(dataset="WGC/GOLD_MONAVG_USD", 
                       start_date="2000-01-01", 
                       end_date="2010-12-31")
    .rename(columns={"Value": "price"})
    .resample("M")
    .last()
)

df.head()

As a confirmation, we can check if the series is stationary -> it is not.

다음 결과는 시계열 데이터가 **비정상성**(non-stationary)이라는 것을 나타냅니다. 아래에서 더 자세히 설명하겠습니다.

1. **ADF (Augmented Dickey-Fuller) 테스트:**
   - ADF 테스트 통계값이 2.41이고, p-value가 1.00입니다.
   - ADF 테스트의 귀무가설은 시계열이 비정상적이라는 것입니다. p-value가 매우 높으므로 귀무가설을 기각할 수 없습니다. 따라서 이 시계열 데이터는 비정상성이라는 결론을 내릴 수 있습니다.

2. **KPSS (Kwiatkowski-Phillips-Schmidt-Shin) 테스트:**
   - KPSS 테스트 통계값이 1.84이고, p-value가 0.01입니다.
   - KPSS 테스트의 귀무가설은 시계열이 정상성이라는 것입니다. p-value가 매우 낮기 때문에 귀무가설을 기각할 수 있습니다. 즉, 시계열 데이터가 정상성이 아니라고 결론지을 수 있습니다.

두 테스트 모두에서 시계열이 정상성을 가지지 않는다는 결론에 도달했기 때문에, 현재의 금 가격 시계열 데이터는 **비정상적**이라고 해석할 수 있습니다. 이를 정상성으로 만들기 위해서는 추가적인 변환(예: 차분 또는 로그 변환 등)이 필요할 수 있습니다.

In [ ]:
fig = test_autocorrelation(df["price"])

우리는 test_autocorrelation 도우미 함수를 사용하여 시계열이 정상적인지 테스트할 수 있습니다. GitHub에서 제공된 노트북에서 이 작업을 수행했으며, 월별 금 가격 시계열은 실제로 정상적이지 않다는 결과가 나왔습니다.

3. Deflate the gold prices (to 2010-12-31 USD values) and plot the results:

우리는 또한 전체 시계열에 대해 동일한 시점이라면, 금 가격을 다른 시점에 맞춰 조정할 수도 있습니다.

3단계에서는 cpi 라이브러리를 사용하여 미국 달러의 인플레이션을 고려해 시계열을 디플레이트(물가를 반영하여 조정)했습니다. 이 라이브러리는 미국 노동통계국(Bureau of Labor Statistics)에서 권장하는 CPI-U 지수에 의존합니다. 이를 작동시키기 위해 datetime.date 클래스의 객체로 된 날짜를 포함하는 인덱스 열을 인위적으로 생성했습니다. inflate 함수는 다음 인수를 받습니다: 
- value—조정하고자 하는 달러 값, 
- year_or_month—달러 값이 속한 날짜, 
- to—선택적으로 값을 조정하려는 날짜(이 인수를 제공하지 않으면 함수는 가장 최근 연도로 조정합니다). 

In [ ]:
DEFL_DATE = date(2010, 12, 31)

df["dt_index"] = pd.to_datetime(df.index)
df["price_deflated"] = df.apply(
    lambda x: cpi.inflate(x["price"], x["dt_index"], DEFL_DATE), 
    axis=1
)

(
    df.loc[:, ["price", "price_deflated"]]
    .plot(title="Gold Price (deflated)")
)

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_6_8", dpi=200)

Figure 6.8: Monthly gold prices and the deflated time series


4. Apply the natural logarithm to the deflated series and plot it together with the rolling metrics:

4단계에서는 자연 로그 함수(np.log)를 모든 값에 적용하여 지수형 추세처럼 보였던 것을 선형으로 변환했습니다. 이 작업은 이미 인플레이션이 반영된 가격에 적용되었습니다. 

In [ ]:
WINDOW = 12
selected_columns = ["price_log", "rolling_mean_log", 
                    "rolling_std_log"]

df["price_log"] = np.log(df.price_deflated)
df["rolling_mean_log"] = df.price_log.rolling(WINDOW) \
                           .mean()
df["rolling_std_log"] = df.price_log.rolling(WINDOW) \
                          .std()

(
    df[selected_columns]
    .plot(title="Gold Price (deflated + logged)", 
          subplots=True)
)

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_6_9", dpi=200)

Figure 6.9: Time series after applying the deflation and natural logarithm, together with its rolling statistics

앞선 플롯에서 볼 수 있듯이, 로그 변환이 제 역할을 했습니다. 즉, 지수 추세를 선형으로 변환했습니다.

5. Use the `test_autocorrelation` (helper function for this chapter) to investigate if the series became stationary:

통계 테스트 결과와 ACF/PACF 플롯을 검토한 후, 디플레이션과 자연 로그 변환만으로는 월별 금 가격 시계열을 정상성으로 만들기에 충분하지 않았다는 결론을 내릴 수 있습니다.

- **ADF (Augmented Dickey-Fuller) 테스트**: ADF 통계값이 1.04이고 p-value가 0.99로 매우 높습니다. ADF 테스트는 귀무가설로 시계열이 비정상성을 가정하므로, 높은 p-value는 시계열이 비정상적임을 의미합니다. 즉, 금 가격 시계열이 정상적이지 않다는 결론에 도달할 수 있습니다.
- **KPSS (Kwiatkowski-Phillips-Schmidt-Shin) 테스트**: KPSS 통계값이 1.93이고 p-value가 0.01로 매우 낮습니다. KPSS 테스트는 귀무가설로 시계열이 정상성을 가정하는데, 낮은 p-value는 시계열이 정상성을 갖지 않는다는 것을 나타냅니다.

이 결과들은 "디플레이션과 자연 로그 변환만으로는 월별 금 가격 시계열을 정상성으로 만들기에 충분하지 않았다"는 번역된 결론과 일치합니다.

In [ ]:
fig = test_autocorrelation(df["price_log"])

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_6_10", dpi=200)

Figure 6.10: The ACF and PACF plots of the transformed time series

6. Apply differencing to the series and plot the results:

마지막 변환으로는 pandas DataFrame의 diff 메서드를 사용하여 시점 t의 값과 t-1의 값 사이의 차이를 계산했습니다(기본 설정은 1차 차분을 의미합니다). period 인수를 변경하여 다른 기간을 지정할 수도 있습니다.

In [ ]:
selected_columns = ["price_log_diff", "roll_mean_log_diff", 
                    "roll_std_log_diff"]

df["price_log_diff"] = df.price_log.diff(1)
df["roll_mean_log_diff"] = df.price_log_diff.rolling(WINDOW) \
                             .mean()
df["roll_std_log_diff"] = df.price_log_diff.rolling(WINDOW) \
                            .std()
df[selected_columns].plot(title="Gold Price (deflated + log + diff)")

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_6_11", dpi=200)

Figure 6.11: Time series after applying three types of transformations, together with its rolling statistics

변환된 금 가격은 정상성을 가진 것처럼 보입니다. 시계열이 0을 중심으로 진동하며, 눈에 띄는 추세 없이 대략 일정한 분산을 보입니다.

7. Test if the series became stationary:

In [ ]:
fig = test_autocorrelation(df["price_log_diff"].dropna())

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_6_12", dpi=200)

Figure 6.12: The ACF and PACF plots of the transformed time series

1차 차분을 적용한 후, 시계열은 5% 유의 수준에서 정상성이 되었으며(두 테스트 모두에 따르면), ACF/PACF 플롯에서 시차 11, 22, 39에서 몇 가지 유의미한 함수 값이 나타났습니다. 이는 어떤 종류의 계절성을 나타내거나 단순히 잘못된 신호일 수 있습니다. 5% 유의 수준을 사용하는 것은 기본 과정에서 자기상관 또는 부분 자기상관을 나타내지 않더라도 95% 신뢰 구간 밖에 5%의 값이 있을 수 있음을 의미합니다.

### There's more

금 가격에는 뚜렷한 계절성이 포함되어 있지 않습니다. 그러나 데이터셋에 계절적 패턴이 나타나는 경우 몇 가지 해결책이 있습니다:

- **차분(differencing)을 통한 조정** 
    - 1차 차분 대신 고차 차분을 사용할 수 있습니다. 예를 들어, 월별 데이터에서 연간 계절성이 존재한다면, `diff(12)`를 사용할 수 있습니다.
   
- **모델링을 통한 조정** 
    - 계절성을 직접 모델링하고, 이를 시계열에서 제거할 수 있습니다. 한 가지 방법은 `seasonal_decompose` 함수나 다른 고급 자동 분해 알고리즘을 사용해 계절적 요소를 추출하는 것입니다. 
    - 이 경우, 가법(additive) 모델을 사용하는 경우에는 계절적 요소를 빼고, 승법(multiplicative) 모델을 사용하는 경우에는 그 요소로 나누면 됩니다. 또 다른 해결책으로는 `np.polyfit()`을 사용하여 선택한 시계열에 가장 적합한 다항식을 맞춘 후, 원본 시계열에서 이를 빼는 방법이 있습니다.

- **Box-Cox 변환**
    - 시계열 데이터에 적용할 수 있는 또 다른 유형의 조정입니다. 이는 다양한 지수 변환 함수를 결합하여 분포를 정규 분포(가우시안 분포)에 더 가깝게 만듭니다. 
    - 우리는 scipy 라이브러리의 `boxcox` 함수를 사용할 수 있으며, 이를 통해 가장 적합한 람다(lambda) 값이 자동으로 찾아집니다. 
    - 주의할 점은, 시계열의 모든 값이 양수여야 한다는 것입니다. 따라서 1차 차분이나 음수 값을 도입할 수 있는 다른 변환을 계산한 후에는 이 변환을 사용할 수 없습니다.

- **pmdarima** 라이브러리(해당 라이브러리에 대한 추가 정보는 이후 레시피에서 다룰 예정입니다)
    - 정상성을 달성하기 위해 시계열을 몇 번 차분해야 하는지 결정하는 두 가지 통계적 테스트를 포함하고 있습니다(또한 계절성을 제거하여 계절 정상성(seasonal stationarity)도 달성). 
    - 다음 테스트들을 사용하여 정상성을 조사할 수 있습니다: ADF, KPSS, 그리고 Phillips–Perron 테스트

In [ ]:
from pmdarima.arima import ndiffs, nsdiffs

In [ ]:
print(f"Suggested # of differences (ADF): {ndiffs(df['price'], test='adf')}")
print(f"Suggested # of differences (KPSS): {ndiffs(df['price'], test='kpss')}")
print(f"Suggested # of differences (PP): {ndiffs(df['price'], test='pp')}")

**KPSS 테스트**의 경우, 우리가 테스트하고자 하는 귀무 가설의 유형을 지정할 수도 있습니다. 기본값은 수준 정상성(null="level")입니다. 테스트 결과는, 더 정확하게 말하자면, 차분의 필요성은 차분 없이 시계열이 정상적이지 않음을 시사합니다.

이 라이브러리에는 계절적 차분을 위한 두 가지 테스트도 포함되어 있습니다:
   - **Osborn, Chui, Smith, and Birchenhall (OCSB)**
   - **Canova-Hansen (CH)**

이 테스트들을 실행하려면 데이터의 주기를 지정해야 합니다. 우리 경우에는 월별 데이터이므로 주기는 12입니다.

In [ ]:
print(f"Suggested # of differences (OSCB): {nsdiffs(df['price'], m=12, test='ocsb')}")
print(f"Suggested # of differences (CH): {nsdiffs(df['price'], m=12, test='ch')}")

## 6.4 Modeling time series with exponential smoothing methods

**지수 평활법**은 고전적인 예측 모델 중 하나로, 과거 관측값의 가중 평균을 사용하여 예측하는 방법입니다. 이 방법에서는 시간이 지남에 따라 최근의 관측값에 더 높은 가중치를 부여합니다. 이 모델은 추세나 계절성이 있는 비정상적인 데이터를 처리하는 데 적합하며, 계산이 간단하고 예측 정확도가 높아 인기가 있습니다.

**ETS(오차, 추세, 계절성) 프레임워크**에서 정의되며, 세 가지 요소(오차, 추세, 계절성)를 더하거나 곱하는 방식으로 결합하거나 생략할 수 있습니다.
1. **단순 지수 평활법 (SES)**: 추세나 계절성이 없는 시계열 데이터에 적합합니다. 
   - 평활화 파라미터 α는 0에서 1 사이의 값을 가지며, 값이 클수록 최근 관측값에 더 큰 가중치를 부여합니다. α = 0일 때는 모든 예측이 훈련 데이터의 평균 값이고, α = 1일 때는 마지막 관측값과 동일한 값을 예측합니다.
   - SES는 평평한 예측 결과를 제공하며, 추세나 계절성이 없는 시계열에만 적합합니다.
2. **Holt의 선형 추세 방법**: SES를 확장한 모델로, 시계열에 추세가 있는 경우 사용할 수 있습니다.
   - 추세가 시간이 지남에 따라 일정하게 유지된다는 단점이 있습니다. 이를 해결하기 위해 **감쇠 파라미터** ϕ를 추가하여 미래에 추세가 일정한 값으로 수렴하도록 합니다.
   - 감쇠 파라미터 ϕ는 일반적으로 0.8에서 0.98 사이의 값을 가지며, ϕ = 1일 때는 감쇠 없이 Holt 모델과 동일해집니다.
3. **Holt-Winters 계절 평활법**: Holt의 방법을 확장한 것으로, 추세와 더불어 계절성까지 고려할 수 있습니다.
   - 이 모델은 계절성을 추가하여 시계열 데이터에서 계절 변동을 처리할 수 있으며, 계절 변동은 **덧셈적** 또는 **곱셈적**으로 적용됩니다.
   - 덧셈적 계절성은 시간이 지남에 따라 변동이 일정하고, 곱셈적 계절성은 시간에 따라 변동이 커지거나 줄어듭니다.

### Getting ready

- 우리는 시계열 분해 레시피에서 사용한 동일한 데이터를 사용할 것입니다.

1. Import the libraries and authenticate:

In [ ]:
import pandas as pd
import nasdaqdatalink
import os 

nasdaqdatalink.ApiConfig.api_key = os.environ['NASDAQ_API_KEY']

2. Download the monthly US unemployment rate from years 2010-2019:

In [ ]:
df = (
    nasdaqdatalink.get(dataset="FRED/UNRATENSA", 
                       start_date="2010-01-01", 
                       end_date="2019-12-31")
    .rename(columns={"Value": "unemp_rate"})
)
df.head()

### How to do it...

- 다음 단계를 실행하여 지수 평활법을 사용해 미국 실업률 예측을 생성하십시오:

1. Import the libraries:

In [ ]:
import pandas as pd
from datetime import date
from statsmodels.tsa.holtwinters import (ExponentialSmoothing, 
                                         SimpleExpSmoothing, 
                                         Holt)

2. Create the train/test split:

In [ ]:
TEST_LENGTH = 12

df.index.freq = "MS"

df_train = df.iloc[:-TEST_LENGTH]
df_test = df.iloc[-TEST_LENGTH:]

3. Fit 2 Simple Exponential Smoothing models and create forecasts:

라이브러리를 불러온 후, 우리는 **SimpleExpSmoothing** 클래스와 그 **fit** 메서드를 사용하여 두 개의 다른 SES 모델을 적합시켰습니다. 모델을 적합시키기 위해 우리는 훈련 데이터만 사용했습니다. 평활화 매개변수(**smoothing_level**) 값을 수동으로 선택할 수도 있었지만, 최선의 방법은 **statsmodels**가 최적의 적합을 위해 이를 자동으로 최적화하도록 하는 것입니다. 이 최적화는 잔차(오차)의 제곱합을 최소화함으로써 이루어집니다. 우리는 **forecast** 메서드를 사용하여 예측을 생성했으며, 이 메서드는 예측할 기간 수를 요구합니다(우리의 경우, 테스트 세트의 길이와 동일합니다). 

In [ ]:
ses_1 = SimpleExpSmoothing(df_train).fit(smoothing_level=0.5)
ses_forecast_1 = ses_1.forecast(TEST_LENGTH)

ses_2 = SimpleExpSmoothing(df_train).fit()
ses_forecast_2 = ses_2.forecast(TEST_LENGTH)

ses_1.params_formatted

In [ ]:
ses_2.summary()

In [ ]:
ses_1.summary()

Figure 6.13: The values of the fitted coefficients for the first SES model

4. Combine the forecasts with the fitted values and plot them:

**4단계**에서는 적합된 모델의 **fittedvalues** 속성을 사용해 적합된 값과 예측값을 pandas DataFrame에 결합하고, 관찰된 실업률과 함께 표시했습니다. 그런 다음 모든 시리즈를 시각화했습니다. 플롯을 더 읽기 쉽게 하기 위해, 우리는 데이터 범위를 훈련 세트의 마지막 2년과 테스트 세트로 제한했습니다.

In [ ]:
ses_1.model.params["smoothing_level"]

In [ ]:
ses_df = df.copy()
ses_df["ses_1"] = ses_1.fittedvalues.append(ses_forecast_1)
ses_df["ses_2"] = ses_2.fittedvalues.append(ses_forecast_2)

# opt_alpha = ses_2.model.params["smoothing_level"]

fig, ax = plt.subplots()
ses_df["2017":].plot(style=["-",":","--"], ax=ax,
                     title="Simple Exponential Smoothing")
labels = [
    "unemp_rate", 
    r"$\alpha={0:.2f}$".format(ses_1.model.params["smoothing_level"]),
    r"$\alpha={0:.2f}$".format(ses_2.model.params["smoothing_level"]), 
]
ax.legend(labels)

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_6_13", dpi=200)

Figure 6.14: Modeling time series using SES

그림 6.14에서 우리는 이 레시피의 서론에서 설명한 SES(단순 지수 평활법)의 특성을 관찰할 수 있습니다. 즉, 예측은 평평한 선입니다. 또한 최적화 절차에서 선택된 최적의 값이 1임을 확인할 수 있습니다. 이러한 값을 선택한 결과는 즉시 알 수 있습니다. 모델의 적합된 선은 실제 관찰된 가격 선을 오른쪽으로 이동시킨 것이며, 예측은 단순히 마지막으로 관찰된 값이 됩니다.

5. Fit 3 variants of Holt's linear trend models and create forecasts:

**5단계**에서는 **Holt** 클래스를 사용하여 Holt의 선형 추세 모델을 적합시켰습니다. 이 클래스는 더 일반적인 **ExponentialSmoothing** 클래스의 래퍼(wrapper)입니다. 기본적으로 모델의 추세는 선형이지만, **exponential=True**를 지정하여 이를 지수적으로 만들 수 있고, **damped_trend=True**를 추가해 감쇠 추세를 적용할 수 있습니다. SES와 마찬가지로, **fit** 메서드를 인수 없이 사용하면 최적의 매개변수 값을 찾기 위한 최적화 절차가 실행됩니다. 

In [ ]:
# Holt's model with linear trend
hs_1 = Holt(df_train).fit()
hs_forecast_1 = hs_1.forecast(TEST_LENGTH)

# Holt's model with exponential trend
hs_2 = Holt(df_train, exponential=True).fit()
# equivalent to ExponentialSmoothing(df_train, trend="mul").fit()
hs_forecast_2 = hs_2.forecast(TEST_LENGTH)

# Holt's model with exponential trend and damping
hs_3 = Holt(df_train, exponential=False, 
            damped_trend=True).fit()
hs_forecast_3 = hs_3.forecast(TEST_LENGTH)

In [ ]:
hs_3.params_formatted

6. Plot the original series together with the models' forecasts:

**6단계**에서는 다시 모든 적합된 값과 예측을 DataFrame에 넣고 결과를 시각화했습니다.

In [ ]:
hs_df = df.copy()
hs_df["hs_1"] = hs_1.fittedvalues.append(hs_forecast_1)
hs_df["hs_2"] = hs_2.fittedvalues.append(hs_forecast_2)
hs_df["hs_3"] = hs_3.fittedvalues.append(hs_forecast_3)

fig, ax = plt.subplots()
hs_df["2017":].plot(style=["-",":","--", "-."], ax=ax,
                    title="Holt's Double Exponential Smoothing")
labels = [
    "unemp_rate", 
    "Linear trend",
    "Exponential trend",
    "Exponential trend (damped)",
]
ax.legend(labels)

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_6_14", dpi=200)

Figure 6.15: Modeling time series using Holt’s Double Exponential Smoothing

우리는 이미 SES 예측과 비교했을 때 선들이 더 이상 평평하지 않다는 점에서 개선을 확인할 수 있습니다. 추가로 언급할 만한 사항은, SES에서는 하나의 파라미터인 알파(α, smoothing_level)만 최적화했지만, 여기서는 베타(β, smoothing_trend)와 필요에 따라 피(ϕ, damping_trend)도 최적화하고 있다는 점입니다.

7. Fit 2 variants of Holt-Winter's Triple Exponential Smoothing models and create forecasts:

**7단계**에서는 두 가지 변형의 Holt-Winters 삼중 지수 평활 모델을 추정했습니다. 이 모델을 위한 별도의 클래스는 없지만, **ExponentialSmoothing** 클래스에 **seasonal** 및 **seasonal_periods** 인수를 추가하여 이를 조정할 수 있습니다. ETS 모델의 분류법에 따르면, 우리는 모델에 덧셈 계절 구성 요소가 있음을 나타내야 합니다. 

In [ ]:
SEASONAL_PERIODS = 12

# Holt-Winters' model with exponential trend
hw_1 = ExponentialSmoothing(df_train, 
                            trend="mul", 
                            seasonal="add", 
                            seasonal_periods=SEASONAL_PERIODS).fit()
hw_forecast_1 = hw_1.forecast(TEST_LENGTH)

# Holt-Winters' model with exponential trend and damping
hw_2 = ExponentialSmoothing(df_train, 
                            trend="mul", 
                            seasonal="add", 
                            seasonal_periods=SEASONAL_PERIODS, 
                            damped_trend=True).fit()
hw_forecast_2 = hw_2.forecast(TEST_LENGTH)

In [ ]:
hw_2.params_formatted

8. Plot the original series together with the models' results:

**8단계**에서는 다시 모든 적합된 값과 예측을 DataFrame에 넣고, 결과를 선 그래프로 시각화했습니다.

In [ ]:
hw_df = df.copy()
hw_df["hw_1"] = hw_1.fittedvalues.append(hw_forecast_1)
hw_df["hw_2"] = hw_2.fittedvalues.append(hw_forecast_2)

fig, ax = plt.subplots()
hw_df["2017":].plot(
    style=["-",":","--"], ax=ax,
    title="Holt-Winters' Triple Exponential Smoothing"
)
phi = hw_2.model.params["damping_trend"]

labels = [
    "unemp_rate", 
    "Seasonal Smoothing",
    f"Seasonal Smoothing (damped with $\phi={phi:.2f}$)"
]
ax.legend(labels)

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_6_15", dpi=200)

Figure 6.16: Modeling time series using Holt-Winters’ Triple Exponential Smoothing

이전 그래프에서 우리는 이제 계절적 패턴이 예측에 통합되었음을 확인할 수 있습니다.

### There's more

이 레시피에서는 다양한 지수 평활법 모델을 적합시켜 월별 실업률을 예측했습니다. 매번 우리가 관심 있는 모델의 종류를 명시했으며, 대부분의 경우에는 statsmodels가 가장 적합한 매개변수를 찾도록 했습니다. 그러나 다른 방식으로 접근할 수도 있습니다. 그 방법은 AutoETS라고 불리는 절차를 사용하는 것입니다. 자세한 설명은 생략하지만, 이 절차의 목표는 우리가 사전에 제공하는 몇 가지 제약 조건을 바탕으로 가장 적합한 ETS 모델을 찾는 것입니다. AutoETS 절차가 어떻게 작동하는지에 대한 더 많은 내용은 See also 섹션에서 언급된 참고 자료를 통해 읽어볼 수 있습니다. AutoETS 절차는 sktime 라이브러리에서 사용할 수 있으며, 이 라이브러리는 scikit-learn에서 영감을 받은 시계열 분석/예측에 중점을 둔 라이브러리/프레임워크입니다. AutoETS 접근 방식을 사용하여 최적의 ETS 모델을 찾기 위해 다음 단계를 실행하십시오.

1. Import the libraries:

In [ ]:
from sktime.forecasting.ets import AutoETS
from sklearn.metrics import mean_absolute_percentage_error

2. Fit the `AutoETS` model:

In [ ]:
auto_ets = AutoETS(auto=True, n_jobs=-1, sp=12)
auto_ets.fit(df_train.to_period())
auto_ets_fcst = auto_ets.predict(fh=list(range(1, 13)))

In [ ]:
auto_ets.summary()

3. Add the model's forecast to the plot of the Holt-Winters' forecasts:

In [ ]:
auto_ets_df = hw_df.to_period().copy()
auto_ets_df["auto_ets"] = (
    auto_ets
    ._fitted_forecaster
    .fittedvalues
    .append(auto_ets_fcst["unemp_rate"])
)

fig, ax = plt.subplots()
auto_ets_df["2017":].plot(
    style=["-",":","--","-."], ax=ax,
    title="Holt-Winters' models vs. AutoETS"
)
labels = [
    "unemp_rate", 
    "Seasonal Smoothing",
    f"Seasonal Smoothing (damped with $\phi={phi:.2f}$)",
    "AutoETS",
]
ax.legend(labels)

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_6_16", dpi=200)

Figure 6.17: The results of the AutoETS forecast plotted over the results of the Holt-Winters’ approach

그림 6.17에서 볼 수 있듯이, Holt-Winters 모델과 AutoETS의 샘플 내 적합(fit)은 매우 유사합니다. 하지만 예측에서는 두 모델이 다소 차이를 보이며, 어느 것이 실업률을 더 잘 예측하는지 판단하기 어렵습니다. 

그래서 다음 단계에서는 **MAPE(평균 절대 백분율 오차)** 를 계산합니다. MAPE는 시계열 예측(및 기타 분야)에서 널리 사용되는 평가 지표입니다.

4. Calculate the MAPEs of the Holt-Winters' forecasts and the ones from AutoETS:

In [ ]:
fcst_dict = {
    "Seasonal Smoothing": hw_forecast_1,
    "Seasonal Smoothing (damped)": hw_forecast_2,
    "AutoETS": auto_ets_fcst,
}

print("MAPEs ----")
for key, value in fcst_dict.items():
    mape = mean_absolute_percentage_error(df_test, value)
    print(f"{key}: {100 * mape:.2f}%")

Holt-Winters 방법과 AutoETS 접근법의 정확도 점수(MAPE로 측정됨)가 매우 유사하다는 것을 확인할 수 있습니다.
- MAPE 값이 클수록 예측의 오차가 크다는 것을 의미합니다. MAPE는 예측이 실제 값과 얼마나 차이가 나는지를 백분율로 나타내므로, 값이 클수록 모델의 예측 성능이 낮다고 볼 수 있습니다.
- AutoETS와 Seasonal Smoothing 모델은 거의 동일한 수준의 예측 정확도를 보여주고 있습니다.
- **Seasonal Smoothing (damped)** 는 다른 두 모델에 비해 예측 오차가 크므로, 해당 데이터에서는 성능이 상대적으로 낮다고 할 수 있습니다.

### See also 

ETS 방법에 대한 추가 정보는 다음 참고 문헌을 참조하십시오:

1. Hyndman, R. J., Akram, Md., & Archibald, 2008. "지수 평활 모델을 위한 허용 가능한 매개변수 공간," *Annals of Statistical Mathematics*, 60(2): 407–426.
2. Hyndman, R. J., Koehler, A.B., Snyder, R.D., & Grose, S., 2002. "지수 평활 방법을 사용한 자동 예측을 위한 상태 공간 프레임워크," *International Journal of Forecasting*, 18(3): 439–454.
3. Hyndman, R. J & Koehler, A. B., 2006. "예측 정확성 측정에 대한 재고," *International Journal of Forecasting*, 22(4): 679-688.
4. Hyndman, R. J., Koehler, A.B., Ord, J.K., & Snyder, R.D. 2008. *Forecasting with Exponential Smoothing: The State Space Approach*, Springer-Verlag. [http://www.exponentialsmoothing.net](http://www.exponentialsmoothing.net).
5. Hyndman, R. J. & Athanasopoulos, G. 2021. *Forecasting: Principles and Practice*, 3판, OTexts: Melbourne, Australia. [OTexts.com/fpp3](http://OTexts.com/fpp3).
6. Winters, P.R. 1960. "지수 가중 이동 평균을 사용한 판매 예측," *Management Science*, 6(3): 324–342. 

## 6.5 Modeling time series with ARIMA class models

ARIMA 모델은 시계열 데이터를 분석하고 예측하는 데 사용되는 통계 모델의 한 유형입니다. 이 모델은 데이터의 자기 상관성을 설명하여 예측을 수행합니다. ARIMA는 **Autoregressive Integrated Moving Average**의 약자로, 보다 간단한 ARMA 모델을 확장한 형태입니다. 이 모델의 추가적인 통합(integration) 요소의 목표는 시계열의 정상성을 보장하는 것입니다. 이는 지수 평활법 모델과 달리 ARIMA 모델은 시계열 데이터가 정상적(stationary)이어야 하기 때문입니다.

아래는 모델의 구성 요소를 간략히 설명한 내용입니다.

**AR (자기회귀) 모델**:
- 이 모델은 관찰치와 그 이전 *p*개의 시점과의 관계를 사용합니다.
- 금융 맥락에서 자기회귀 모델은 모멘텀 효과와 평균 회귀 효과를 설명하려고 시도합니다.

**I (통합)**:
- 여기서 통합은 원래 시계열 데이터를 차분(differencing)하는 것을 의미하며, 이는 이전 시점의 값을 현재 시점의 값에서 빼서 정상성을 만드는 것을 의미합니다.
- 통합을 담당하는 매개변수는 *d* (차분 차수)이며, 차분을 적용해야 하는 횟수를 나타냅니다.

**MA (이동 평균) 모델**:
- 이 모델은 관찰치와 최근 *q* 개의 관측값에 대해 발생한 백색잡음(white noise)과의 관계를 사용합니다.
- 금융 맥락에서 이동 평균 모델은 잔차에서 관찰된 예측 불가능한 충격(자연 재해, 특정 기업과 관련된 속보 등)이 시계열 데이터에 미치는 영향을 설명하려고 합니다.

- MA 모델에서 백색잡음 항은 관찰할 수 없습니다. 이 때문에 **최소 제곱법 (OLS)** 을 사용하여 ARIMA 모델을 맞출 수 없습니다. 대신, **MLE (최대우도추정법)** 과 같은 반복적인 추정 방법을 사용해야 합니다.

이 모든 구성 요소는 함께 결합되어 **ARIMA (p,d,q)** 라는 일반적인 표기법으로 직접 지정됩니다. 일반적으로 ARIMA 매개변수 값을 가능한 한 작게 유지하여 불필요한 복잡성을 피하고, 훈련 데이터에 대한 과적합을 방지하는 것이 좋습니다. 하나의 경험적 규칙으로는 *d* <= 2, *p* 와 *q* 는 5를 넘지 않도록 유지하는 것입니다. 또한 AR 또는 MA 중 하나가 모델에서 지배적인 역할을 할 가능성이 크며, 다른 하나는 비교적 작은 값을 가질 가능성이 높습니다.

---

**ARIMA 모델은 매우 유연하며, 적절한 하이퍼파라미터 설정을 통해 다음과 같은 특수한 경우들을 얻을 수 있습니다:**

- **ARIMA (0,0,0):** 백색잡음(White noise)
- **ARIMA (0,1,0) 상수 없이:** 랜덤 워크(Random walk)
- **ARIMA (p,0,q):** ARMA(p, q)
- **ARIMA (p,0,0):** AR(p) 모델
- **ARIMA (0,0,q):** MA(q) 모델
- **ARIMA (0,1,2):** 감쇠된 홀트 모델(Damped Holt’s model)
- **ARIMA (0,1,1) 상수 없이:** SES 모델
- **ARIMA (0,2,2):** 홀트의 선형 방식(Holt’s linear method) (가법 오류)

---

ARIMA 모델은 여전히 산업계에서 매우 인기가 있으며, 특히 소규모 데이터셋을 다룰 때 단기 예측에서 거의 최신 성능에 가까운 결과를 제공합니다. 이러한 경우, 더 발전된 머신러닝 및 딥러닝 모델들은 그 진정한 힘을 발휘하기 어렵습니다.

ARIMA 모델의 금융 맥락에서 알려진 약점 중 하나는 대부분의 금융 자산에서 관찰되는 변동성 군집(volatility clustering)을 포착하지 못한다는 것입니다.

이 레시피에서는 ARIMA 모델을 올바르게 추정하는 데 필요한 모든 단계를 살펴보고, 데이터에 적합한지 확인하는 방법을 배울 것입니다. 이 예시에서는 2010년부터 2019년까지의 미국 월간 실업률 데이터를 다시 사용할 것입니다.

### Getting ready

We will use the same data that we used in the Time series decomposition recipe.

1. Import the libraries and authenticate:

In [ ]:
import pandas as pd
import nasdaqdatalink
import os 

nasdaqdatalink.ApiConfig.api_key = os.environ['NASDAQ_API_KEY']

2. Download the monthly US unemployment rate from years 2010-2019:

In [ ]:
df = (
    nasdaqdatalink.get(dataset="FRED/UNRATENSA", 
                       start_date="2010-01-01", 
                       end_date="2019-12-31")
    .rename(columns={"Value": "unemp_rate"})
)

# to hide the warnings of `statsmodels`
df.index.freq = "MS"

df.head()

### How to do it...

1. Import the libraries:

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from chapter_6_utils import test_autocorrelation
from sklearn.metrics import mean_absolute_percentage_error

2. Create the train/test split:

In [ ]:
TEST_LENGTH = 12
df_train = df.iloc[:-TEST_LENGTH]
df_test = df.iloc[-TEST_LENGTH:]

3. Apply the log transformation and calculate the first differences:

2단계에서 훈련 데이터와 테스트 데이터를 생성한 후, 로그 변환과 1차 차분(first differences)을 훈련 데이터에 적용했습니다. 주어진 시계열에 여러 번 차분을 적용하려면 **np.diff** 함수를 사용하는 것이 좋습니다. 이 함수는 재귀적 차분을 구현합니다. DataFrame/Series의 **diff** 메서드를 사용할 때 **periods > 1**로 설정하면 현재 관측치와 해당 주기만큼 이전의 관측치 사이의 차이를 계산합니다.  

In [ ]:
df_train["unemp_rate_log"] = np.log(df_train["unemp_rate"])
df_train["first_diff"] = df_train["unemp_rate_log"].diff()

df_train.plot(subplots=True, 
              title="Original vs transformed series")

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_6_17", dpi=200)

Figure 6.18: Applying transformations to achieve stationarity

4. Test the stationarity of the differenced series:

4단계에서는 로그 변환된 시계열의 1차 차분에 대해 정상성을 테스트했습니다. 이를 위해 **test_autocorrelation**이라는 사용자 정의 함수를 사용했습니다. 통계적 테스트 결과를 살펴본 결과, 해당 시계열은 5% 유의 수준에서 정상적임을 확인할 수 있었습니다. 또한, ACF/PACF 플롯을 보면 연간 계절 패턴이 명확히 보입니다(12 및 24 지연에서).  

In [ ]:
fig = test_autocorrelation(df_train["first_diff"].dropna())

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_6_18", dpi=200)

Figure 6.19: The autocorrelation plots of the first differences of the log transformed series

5. Fit two different ARIMA models and print their summaries:

5단계에서 우리는 두 개의 ARIMA 모델을 적합시켰습니다: **ARIMA(1,1,1)** 및 **ARIMA(2,1,2)**. 먼저, 시계열은 1차 차분 후 정상적으로 변했으므로, 통합 차수는 **d=1**로 설정했습니다. 일반적으로 **p**와 **q** 값을 결정할 때는 다음과 같은 "규칙"을 사용할 수 있습니다.

**AR 모델의 차수(p)를 식별하는 방법:**
- **ACF**는 **p** 지연까지 유의한 자기상관을 나타내고, 그 이후에는 점차 사라집니다.
- **PACF**는 관측값과 그 지연 사이의 직접적인 관계만 설명하므로, **p** 지연 이후에는 유의한 상관관계가 없어야 합니다.

**MA 모델의 차수(q)를 식별하는 방법:**
- **PACF**는 **q** 지연까지 유의한 자기상관을 나타내고, 그 이후에는 점차 사라집니다.
- **ACF**는 **q** 지연까지 유의한 자기상관 계수를 나타내며, 그 후에는 급격히 감소합니다.

ARIMA 모델의 차수를 수동으로 조정할 때, Hyndman과 Athanasopoulos(2018)는 **p**와 **q**가 모두 양수일 경우 **ACF/PACF** 플롯이 ARIMA 모델의 구성을 결정하는 데 유용하지 않을 수 있다고 경고했습니다. 다음 레시피에서는 ARIMA 하이퍼파라미터의 최적 값을 결정하는 자동화된 접근 방식을 소개할 것입니다.  

In [ ]:
arima_111 = ARIMA(
    df_train["unemp_rate_log"], order=(1, 1, 1)
).fit()
arima_111.summary()

Figure 6.20: The summary of the fitted ARIMA(1,1,1) model

In [ ]:
arima_212 = ARIMA(
    df_train["unemp_rate_log"], order=(2, 1, 2)
).fit()
arima_212.summary()

Figure 6.21: The summary of the fitted ARIMA(2,1,2) model

6. Combine the fitted values with the predictions:

6단계에서는 원래의 시계열 데이터와 두 모델의 예측 값을 결합했습니다. ARIMA 모델에서 적합된 값을 추출하고 2019년의 예측 값을 시계열 끝에 추가했습니다. 모델을 로그 변환된 시계열에 적합시켰기 때문에, **np.exp**를 사용하여 변환을 역으로 적용해야 했습니다. 0 값을 가질 수 있는 시계열을 다룰 때는 **np.log1p**와 **np.exp1m**을 사용하는 것이 더 안전합니다. 이렇게 하면 0에 대한 로그를 계산할 때 발생할 수 있는 오류를 방지할 수 있습니다.  

In [ ]:
df["pred_111_log"] = (
    arima_111
    .fittedvalues
    .append(arima_111.forecast(TEST_LENGTH))
)
df["pred_111"] = np.exp(df["pred_111_log"])

df["pred_212_log"] = (
    arima_212
    .fittedvalues
    .append(arima_212.forecast(TEST_LENGTH))
)
df["pred_212"] = np.exp(df["pred_212_log"])
df

Figure 6.22: The predictions from the ARIMA models—raw and transformed back to the original scale

In [ ]:
#  The number of initial periods during which the loglikelihood is not recorded. Default is 0.
arima_111.loglikelihood_burn

In [ ]:
arima_111.nobs_diffuse

7. Plot the forecasts and calculate the MAPEs:

7단계에서는 예측을 시각화하고, 평균 절대 백분율 오차(MAPE)를 계산했습니다. **ARIMA(2,1,2)** 모델은 단순한 **ARIMA(1,1,1)** 모델보다 훨씬 더 나은 예측을 제공했습니다.  

In [ ]:
(
    df[["unemp_rate", "pred_111", "pred_212"]]
    .iloc[1:]
    .plot(title="ARIMA forecast of the US unemployment rate")
);

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_6_21", dpi=200)

Figure 6.23: The forecast and the fitted values from the two ARIMA models

Now we also zoom into the test set, to clearly see the forecasts:

In [ ]:
(
    df[["unemp_rate", "pred_111", "pred_212"]]
    .iloc[-TEST_LENGTH:]
    .plot(title="Zooming in on the out-of-sample forecast")
);

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_6_22", dpi=200)

Figure 6.24: The forecast from the two ARIMA models

In Figure 6.24, we can see that the forecast of the ARIMA(1,1,1) is virtually a straight line, while ARIMA(2,1,2) did a better job at capturing the pattern of the original series.

Now we calculate the MAPEs:

In [ ]:
mape_111 = mean_absolute_percentage_error(
    df["unemp_rate"].iloc[-TEST_LENGTH:], 
    df["pred_111"].iloc[-TEST_LENGTH:]
)

mape_212 = mean_absolute_percentage_error(
    df["unemp_rate"].iloc[-TEST_LENGTH:], 
    df["pred_212"].iloc[-TEST_LENGTH:]
)

print(f"MAPE of ARIMA(1,1,1): {100 * mape_111:.2f}%")
print(f"MAPE of ARIMA(2,1,2): {100 * mape_212:.2f}%")

8. Extract the forecast with the corresponding confidence intervals and plot them all together:

8단계에서는 적합된 ARIMA 모델의 **get_forecast** 메서드와 **summary_frame** 메서드를 연결하여 예측 값과 해당 신뢰 구간을 얻었습니다. **forecast** 메서드는 점 예측 값만 반환하고 추가 정보를 제공하지 않기 때문에 **get_forecast** 메서드를 사용해야 했습니다. 마지막으로, 열 이름을 변경한 후 원래 시계열과 함께 시각화했습니다.


1. **"fcst" (Forecast):**
   - **의미**: ARIMA(2,1,2) 모델이 생성한 예측 값입니다. `arima_212.get_forecast(TEST_LENGTH)`가 테스트 데이터 기간 동안의 예측을 반환하며, 이 예측 값이 `"fcst"` 컬럼에 저장됩니다.
   - **내용**: 변환되지 않은 로그 값의 예측이므로, 이를 시각화할 때는 로그 변환을 역으로 적용해 실제 값으로 변환할 필요가 있습니다(코드에서 `np.exp`가 이를 처리함).
2. **"fcst_se" (Forecast Standard Error):**
   - **의미**: 예측 값의 표준 오차입니다. 모델이 예측한 값의 불확실성을 나타내며, 예측의 신뢰도를 평가할 때 중요한 역할을 합니다.
   - **내용**: 표준 오차가 작을수록 예측의 신뢰도가 높고, 표준 오차가 클수록 예측의 불확실성이 크다는 것을 의미합니다.
3. **"ci_lower" (Confidence Interval Lower Bound):**
   - **의미**: 예측 값의 **하한 신뢰 구간**입니다. 주어진 신뢰 수준(보통 95% 신뢰 구간 기준)에서 예측 값의 가장 낮은 값이 될 가능성이 있는 값을 나타냅니다.
   - **내용**: 예측의 불확실성을 표현하며, 예측 값이 이 구간 안에 있을 가능성이 큽니다. 하한 신뢰 구간 값은 모델이 예측한 값보다 작을 수 있습니다.
4. **"ci_upper" (Confidence Interval Upper Bound):**
   - **의미**: 예측 값의 **상한 신뢰 구간**입니다. 주어진 신뢰 수준(보통 95% 신뢰 구간 기준)에서 예측 값의 가장 높은 값이 될 가능성이 있는 값을 나타냅니다.
   - **내용**: 상한 신뢰 구간 값은 예측 값보다 클 수 있으며, 예측의 불확실성을 상한으로 나타냅니다.

In [ ]:
preds_df = arima_212.get_forecast(TEST_LENGTH).summary_frame()
preds_df.columns = ["fcst", "fcst_se", "ci_lower", "ci_upper"]
plot_df = df_test[["unemp_rate"]].join(np.exp(preds_df))
plot_df.head()

In [ ]:
fig, ax = plt.subplots()

(
    plot_df[["unemp_rate", "fcst"]]
    .plot(ax=ax,
          title="ARIMA(2,1,2) forecast with confidence intervals")
)

ax.fill_between(plot_df.index,
                plot_df["ci_lower"],
                plot_df["ci_upper"],
                alpha=0.3, 
                facecolor="g")

ax.legend(loc="upper left");

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_6_23", dpi=200)

Figure 6.25: The forecast from the ARIMA(2,1,2) model together with its confidence intervals

우리는 예측 값이 관측된 값의 형태를 따르고 있음을 확인할 수 있습니다. 또한, 전형적인 원뿔 모양의 신뢰 구간 패턴도 볼 수 있습니다. 예측 범위가 길어질수록 신뢰 구간이 넓어지며, 이는 증가하는 불확실성과 대응됩니다.

### There's more

1. Plot diagnostic plots for the residuals of the fitted ARIMA(2,1,2) model:

In [ ]:
arima_212.plot_diagnostics(figsize=(18, 14), lags=25)

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_6_24", dpi=200)

Figure 6.26: The diagnostics plot of the fitted ARIMA(2,1,2) model

- **표준화된 잔차의 시간에 따른 변화 (왼쪽 위)**: 잔차는 백색 잡음처럼 행동해야 하며, 즉 명확한 패턴이 없어야 합니다. 또한, 잔차는 평균이 0이고 분산이 일정해야 합니다. 우리의 경우, 음수 값이 양수 값보다 더 많이 보이므로, 평균도 음수일 가능성이 높습니다.
- **히스토그램과 KDE 추정치 (오른쪽 위)**: 잔차의 KDE 곡선은 표준 정규 분포(표시된 N(0,1))와 매우 유사해야 합니다. 하지만 우리 모델의 경우, 분포가 음수 값 쪽으로 치우쳐 있음을 알 수 있습니다.
- **Q-Q 플롯 (왼쪽 아래)**: 대부분의 데이터 포인트가 직선 위에 있어야 합니다. 이는 이론적 분포(표준 정규 분포)의 분위수와 경험적 분위수가 일치한다는 것을 나타냅니다. 대각선에서 크게 벗어난 값들은 경험적 분포가 왜곡되었음을 의미합니다.
- **상관도표 (Correlogram) (오른쪽 아래)**: 여기서는 잔차의 자기상관 함수 플롯을 보고 있습니다. 잘 적합된 ARIMA 모델의 잔차는 자기상관이 없어야 합니다. 하지만 우리 경우, 12와 24 지연에서 잔차가 명확히 상관되어 있는 것을 볼 수 있습니다. 이는 모델이 데이터의 계절적 패턴을 제대로 포착하지 못하고 있음을 암시합니다.

잔차의 자기상관을 계속 조사하기 위해, **Ljung-Box 검정**을 사용하여 자기상관이 없는지 확인할 수 있습니다. 이를 위해 적합된 ARIMA 모델의 **test_serial_correlation** 메서드를 사용할 수 있습니다. 또는 **statsmodels**의 **acorr_ljungbox** 함수를 사용할 수 있습니다.

2. Apply the Ljung-Box's test for no autocorrelation in the residuals and plot the results:

In [ ]:
ljung_box_results = arima_212.test_serial_correlation(method="ljungbox")
ljung_box_pvals = ljung_box_results[0][1]

fig, ax = plt.subplots(1, figsize=[16, 5])
sns.scatterplot(x=range(len(ljung_box_pvals)), 
                y=ljung_box_pvals, 
                ax=ax)
ax.axhline(0.05, ls="--", c="r")
ax.set(title="Ljung-Box test's results",
       xlabel="Lag",
       ylabel="p-value")

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_6_25", dpi=200)

Figure 6.27: The results of the Ljung-Box test for no autocorrelation in the residuals

In [ ]:
# alternative way to get the same test results
from statsmodels.stats.diagnostic import acorr_ljungbox
ljung_box_results_df = acorr_ljungbox(arima_212.resid[1:])
ljung_box_results_df

The first residual of the fitted ARIMA/ARMA model is equal to the first observation of the time series. For more information, please see the following:
* https://stats.stackexchange.com/questions/202903/start-up-values-for-the-kalman-filter/221723#221723

In [ ]:
df_train

In [ ]:
arima_212.resid

Other available tests:

In [ ]:
arima_212.test_normality(method="jarquebera")

In [ ]:
arima_212.test_heteroskedasticity(method="breakvar")

### See also 

Please see the following references for more information on fitting ARIMA models and helpful sets of rules for manually picking up the correct orders of the models:
- https://online.stat.psu.edu/stat510/lesson/3/3.1 
- https://people.duke.edu/~rnau/arimrule.htm

For more information on the Ljung-Box test: 
- https://robjhyndman.com/hyndsight/ljung-box-test/



## 6.6 Finding the best-fitting ARIMA model with auto-ARIMA

이전 레시피에서 본 것처럼 ARIMA 모델의 성능은 선택한 하이퍼파라미터(p, d, q)에 따라 크게 달라집니다. 우리는 직관이나 통계 테스트, ACF/PACF 그래프를 기반으로 최선의 선택을 할 수 있지만, 실제로는 이를 수행하는 것이 쉽지 않습니다.

그래서 이번 레시피에서는 auto-ARIMA를 소개합니다. 이는 ARIMA 클래스 모델(ARIMAX 및 SARIMA와 같은 변형을 포함)의 최적 하이퍼파라미터를 찾는 자동화된 방법입니다.

알고리즘의 기술적 세부 사항에 깊이 들어가지 않고, 먼저 KPSS 테스트를 사용하여 차분의 수를 결정합니다. 그런 다음 알고리즘은 스텝 와이즈 탐색(stepwise search)을 사용하여 모델 공간을 탐색하며 더 적합한 모델을 찾습니다. 모델 비교에 자주 사용되는 평가 지표는 **Akaike 정보 기준(AIC)** 입니다. 이 메트릭은 모델의 적합도와 단순성 사이의 절충점을 제공합니다. AIC는 과적합과 언더피팅의 위험을 처리하는 데 도움을 줍니다. 여러 모델을 비교할 때, AIC 값이 낮을수록 더 좋은 모델입니다. auto-ARIMA 절차에 대한 자세한 설명은 '참고 자료' 섹션을 참고하십시오.

auto-ARIMA 프레임워크는 ARIMA 모델의 확장 모델과도 잘 작동합니다:
- **ARIMAX**: 모델에 외생 변수(들)을 추가합니다.
- **SARIMA (Seasonal ARIMA)**: 시계열의 계절성을 고려하도록 ARIMA를 확장합니다. 전체 사양은 $SARIMA(p,d,q)(P,D,Q)m$ 이며, 대문자 파라미터는 계절 구성 요소를 나타냅니다. 'm'은 계절성 주기를 나타냅니다.

이번 레시피에서는 2010년부터 2019년까지의 미국 월간 실업률 데이터를 다시 사용할 예정입니다.

### Getting ready

우리는 **시계열 분해** 레시피에서 사용한 동일한 데이터를 사용할 것입니다.

1. Import the libraries and authenticate:

In [ ]:
import pandas as pd
import nasdaqdatalink
import os 

nasdaqdatalink.ApiConfig.api_key = os.environ['NASDAQ_API_KEY']

2. Download the monthly US unemployment rate from years 2010-2019:

In [ ]:
df = (
    nasdaqdatalink.get(dataset="FRED/UNRATENSA", 
                       start_date="2010-01-01", 
                       end_date="2019-12-31")
    .rename(columns={"Value": "unemp_rate"})
)

# to hide the warnings of `statsmodels`
df.index.freq = "MS"

df.head()

### How to do it...

1. Import the libraries:

In [ ]:
import pandas as pd
import pmdarima as pm
from sklearn.metrics import mean_absolute_percentage_error

2. Create the train/test split:

라이브러리를 가져온 후, 우리는 이전 레시피에서 했던 것처럼 훈련 및 테스트 세트를 생성했습니다.

In [ ]:
TEST_LENGTH = 12
df_train = df.iloc[:-TEST_LENGTH]
df_test = df.iloc[-TEST_LENGTH:]

3. Find the best hyperparameters of the ARIMA model using the auto-ARIMA procedure:

3단계에서, **auto_arima** 함수를 사용하여 ARIMA 모델의 최적 하이퍼파라미터를 찾았습니다. 사용 시 다음과 같이 설정했습니다:
- 정적성(stationarity) 테스트로 KPSS 테스트 대신 Augmented Dickey-Fuller 테스트를 사용하고자 했습니다.
- SARIMA 대신 ARIMA 모델을 적합시키기 위해 계절성을 껐습니다.
- 절편 없이 모델을 적합하고자 했습니다. 이는 **statsmodels**에서 ARIMA를 추정할 때 기본 설정입니다(ARIMA 클래스의 **trend** 인수에서).
- 하이퍼파라미터를 식별하기 위해 단계적(stepwise) 알고리즘을 사용하고자 했습니다. 이 설정을 **False**로 지정하면, **scikit-learn**의 GridSearchCV 클래스와 유사하게 가능한 모든 하이퍼파라미터 조합을 시도하는 총망라된 그리드 탐색을 실행합니다. 이 경우, 몇 개의 모델이 병렬로 적합될 수 있는지 **n_jobs** 인수를 통해 지정할 수 있습니다.


다른 다양한 설정들도 실험해 볼 수 있습니다. 예를 들면:
- 검색을 위한 하이퍼파라미터의 시작값을 선택합니다.
- 검색에서 파라미터의 최대값을 제한합니다.
- 차분(계절적 차분 포함) 수를 결정하기 위한 다른 통계적 테스트를 선택합니다.
- 샘플 외 평가 기간( **out_of_sample_size** )을 선택합니다. 이를 통해 알고리즘이 특정 시점까지의 데이터로 모델을 적합시키고(마지막 관측값에서 **out_of_sample_size**를 뺀 값) 보유된 세트에서 평가할 수 있습니다. 모델을 선택할 때 예측 성능이 더 중요한 경우 이 방법이 유용할 수 있습니다.
- 모델 적합의 최대 시간 또는 시도할 하이퍼파라미터 조합의 최대 수를 제한할 수 있습니다. 계절적 모델을 주별 데이터와 같은 더 세밀한 데이터에 적합할 때 유용하며, 이러한 경우 적합하는 데 시간이 오래 걸리는 경향이 있습니다.


우리가 trace=True를 설정했기 때문에, 절차 중 적합된 모델에 대한 다음과 같은 정보를 확인할 수 있었습니다:

In [ ]:
auto_arima = pm.auto_arima(df_train,
                           test="adf",
                           seasonal=False,
                           with_intercept=False,
                           stepwise=True,
                           suppress_warnings=True,
                           trace=True)
                            
auto_arima.summary()

Figure 6.28: The summary of the best-fitting ARIMA model, as identified using the auto-ARIMA procedure

절차에 따르면 최적의 ARIMA 모델은 ARIMA(2,1,2)입니다. 하지만 Figure 6.28과 Figure 6.21의 결과는 다릅니다. 이는 후자의 경우 ARIMA(2,1,2) 모델을 로그 변환된 시계열에 맞췄지만, 이번 레시피에서는 로그 변환을 적용하지 않았기 때문입니다.

statsmodels 라이브러리로 추정된 ARIMA 모델과 유사하게, pmdarima (실제로는 statsmodels를 감싸는 래퍼)에서도 plot_diagnostics 메서드를 사용하여 모델의 잔차를 분석할 수 있습니다.

In [ ]:
auto_arima.plot_diagnostics(figsize=(18, 14), lags=25)

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_6_27", dpi=200)

Figure 6.29: The diagnostic plots of the best-fitting ARIMA model

Figure 6.26의 진단 플롯과 유사하게, 이 ARIMA(2,1,2) 모델도 연간 계절 패턴을 포착하는 데 어려움을 겪고 있습니다. 이는 상관도(correlogram)에서 명확히 볼 수 있습니다.

이 이미지에서 보이는 진단 플롯을 바탕으로 ARIMA 모델의 성능을 분석할 수 있습니다. 각 플롯의 의미를 설명하자면:

1. **표준화된 잔차(Standardized Residuals) (왼쪽 상단)**: 
   - 이 그래프는 모델에서 나온 잔차(예측값과 실제값의 차이)를 보여줍니다. 잔차가 일정한 분포를 따르는지 확인할 수 있습니다. 이 경우, 잔차가 일정한 패턴을 따르지 않고 임의로 분포된 것으로 보입니다. 이상치나 특정 패턴이 없는 것이 바람직합니다.

2. **히스토그램 및 추정 밀도(Histogram plus Estimated Density) (오른쪽 상단)**: 
   - 잔차의 분포를 히스토그램으로 보여주며, 추정된 밀도(KDE)와 표준 정규 분포(N(0,1))와 비교한 것입니다. 이 플롯에서 잔차가 정규 분포를 따르는지 여부를 시각적으로 확인할 수 있습니다. 이 경우, 대체로 정규 분포와 유사한 형태를 보이지만, 약간의 차이가 있습니다.

3. **정규 Q-Q 플롯(Normal Q-Q Plot) (왼쪽 하단)**: 
   - 이 플롯은 잔차가 이론적인 정규 분포를 얼마나 잘 따르는지를 나타냅니다. 모든 점이 주황색 선(이론적 정규 분포) 위에 위치한다면, 잔차는 정규 분포를 잘 따르고 있는 것입니다. 여기서는 대부분의 점이 선에 근접해 있지만, 극단적인 값에서는 약간 벗어난 것을 볼 수 있습니다. 이는 일부 이상치가 있음을 시사합니다.

4. **상관도(Correlogram) (오른쪽 하단)**: 
   - 이 플롯은 잔차의 자기상관성을 보여줍니다. 자기상관이란 잔차 간에 시간이 지나도 관계가 있는지 확인하는 것인데, 잔차가 자기상관을 가지고 있으면 모델이 제대로 데이터를 설명하지 못했음을 의미합니다. 이 그래프에서 파란색 막대들이 95% 신뢰 구간을 벗어나면 자기상관성이 있다는 것을 의미합니다. 이 경우 일부 구간에서 상관성이 있는 것을 확인할 수 있습니다. 이는 모델이 데이터를 완벽하게 설명하지 못한다는 신호일 수 있습니다.

따라서 이 ARIMA(2,1,2) 모델은 대체로 정상적으로 작동하지만, 몇 가지 자기상관성이 남아 있고 일부 이상치가 존재하는 것으로 보입니다. 특히 상관도 플롯에서 연간 계절 패턴을 완벽하게 포착하지 못했음을 확인할 수 있습니다.

추가적으로, 모델을 개선하려면 더 많은 데이터를 사용하거나 계절성을 더 잘 반영할 수 있는 SARIMA와 같은 다른 모델을 시도해 볼 수 있습니다.

4. Find the best hyperparameters of a SARIMA model using the auto-ARIMA procedure:

4단계에서, **auto_arima** 함수를 사용하여 최적의 SARIMA 모델을 찾았습니다. 이를 위해 **seasonal=True**로 설정하고 월별 데이터로 작업하고 있다는 것을 나타내기 위해 **m=12**를 지정했습니다.

**추가 정보**

우리는 **pmdarima** 라이브러리에서 auto-ARIMA 프레임워크를 사용하여 더 복잡한 모델이나 전체 파이프라인(타겟 변수를 변환하거나 새 기능을 추가하는 등)을 추정할 수 있습니다. 이 섹션에서는 그렇게 하는 방법을 설명합니다.

In [ ]:
auto_sarima = pm.auto_arima(df_train,
                            test="adf",
                            seasonal=True,
                            m=12,
                            with_intercept=False,
                            stepwise=True,
                            suppress_warnings=True,
                            trace=True)
auto_sarima.summary()

Figure 6.30: The summary of the best-fitting SARIMA model, as identified using the auto-ARIMA procedure

Please refer to the following SO question to see why the procedure returns `AIC=inf` for some model specifications:
* https://stats.stackexchange.com/questions/160612/auto-arima-doesnt-calculate-aic-values-for-the-majority-of-models

In [ ]:
auto_sarima.plot_diagnostics(figsize=(18, 14), lags=25);

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_6_29", dpi=200)

Figure 6.31: The diagnostic plots of the best-fitting SARIMA model

이미지에 나오는 SARIMA 모델의 진단 플롯을 분석하고 설명하자면, 이 플롯들은 ARIMA 모델과 마찬가지로 잔차가 모델의 가정을 얼마나 잘 따르는지를 시각적으로 보여줍니다. 각 플롯을 차례대로 설명해보겠습니다:

1. **표준화된 잔차(Standardized Residuals) (왼쪽 상단)**:
   - SARIMA 모델의 잔차가 시간에 따라 일정한 패턴을 따르지 않고 분포되는지를 보여줍니다. 이 플롯에서는 잔차가 무작위로 분포되어 있어 모델이 적절하게 데이터를 설명하고 있다는 것을 시사합니다. 여기서는 큰 이상치나 특정 패턴이 나타나지 않으므로, SARIMA 모델이 데이터를 비교적 잘 적합하고 있음을 보여줍니다.

2. **히스토그램 및 추정 밀도(Histogram plus Estimated Density) (오른쪽 상단)**:
   - 잔차의 분포를 히스토그램으로 보여주며, 추정된 밀도(KDE)와 표준 정규 분포(N(0,1))와 비교한 것입니다. 이 플롯에서 잔차가 거의 정규 분포를 따르고 있음을 확인할 수 있습니다. SARIMA 모델의 잔차는 거의 정규 분포를 따르는 것으로 보이며, 이는 좋은 징후입니다.

3. **정규 Q-Q 플롯(Normal Q-Q Plot) (왼쪽 하단)**:
   - 이 플롯은 잔차가 이론적인 정규 분포를 얼마나 잘 따르는지를 나타냅니다. 모든 점이 주황색 선(이론적 정규 분포) 위에 가깝게 위치하면 잔차가 정규 분포를 잘 따르고 있는 것입니다. 이 SARIMA 모델에서는 대부분의 점이 주황색 선 위에 위치해 있으며, 이는 잔차가 정규 분포를 잘 따르고 있음을 나타냅니다. 하지만 극단적인 몇몇 값들은 선에서 약간 벗어난 것을 볼 수 있으며, 이는 약간의 이상치를 시사합니다.

4. **상관도(Correlogram) (오른쪽 하단)**:
   - 이 플롯은 잔차의 자기상관성을 보여줍니다. SARIMA 모델의 경우 대부분의 상관 관계가 95% 신뢰 구간 내에 있으며, 이는 잔차 간의 자기상관성이 거의 없음을 나타냅니다. 자기상관성이 없다는 것은 모델이 데이터를 잘 설명하고 있다는 것을 의미합니다.

**전체 요약**:
이 SARIMA 모델의 경우, ARIMA(2,1,2) 모델과 비교했을 때 더 나은 적합도를 보이는 것으로 보입니다. 이는 잔차의 분포가 정규 분포를 잘 따르고, 자기상관성도 거의 없는 것을 통해 확인할 수 있습니다. 따라서 SARIMA 모델이 데이터의 계절성을 더 잘 반영하여 예측 성능을 향상시켰음을 시사합니다.

5. Calculate the forecasts from the two models and plot them:

In [ ]:
df_test["auto_arima"] = auto_arima.predict(TEST_LENGTH)
df_test["auto_sarima"] = auto_sarima.predict(TEST_LENGTH)
df_test.plot(title="Forecasts of the best ARIMA/SARIMA models");

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_6_30", dpi=200)

Figure 6.32: The forecasts from the ARIMA and SARIMA models identified using the auto-ARIMA procedure

이 그래프는 ARIMA와 SARIMA 모델을 사용하여 예측한 실업률(unemp_rate)의 변화를 시각화한 것입니다. 그래프에 대한 설명은 다음과 같습니다.

- **파란색 선 (unemp_rate)**: 실제 실업률 데이터입니다. 2019년 동안의 월별 실업률 변화를 보여줍니다.
- **주황색 선 (auto_arima)**: 자동으로 ARIMA 모델을 사용하여 예측한 실업률 값입니다. 이 선은 실제 데이터와 비교해보면 일부 구간에서 실제 값과 다르게 예측되고 있음을 볼 수 있습니다.
- **녹색 선 (auto_sarima)**: 자동으로 SARIMA 모델을 사용하여 예측한 실업률 값입니다. SARIMA 모델의 경우, ARIMA 모델에 비해 더 정확하게 실제 실업률 데이터를 따라가고 있음을 볼 수 있습니다.
  
따라서 이 그래프에서 볼 수 있듯이, SARIMA 모델이 ARIMA 모델보다 2019년 실업률 예측에서 더 나은 성능을 보이고 있습니다. 이는 계절적 요소가 있는 데이터에서 SARIMA 모델이 더 적합하다는 것을 시사합니다.

In [ ]:
mape_auto_arima = mean_absolute_percentage_error(
    df_test["unemp_rate"], 
    df_test["auto_arima"]
)

mape_auto_sarima = mean_absolute_percentage_error(
    df_test["unemp_rate"], 
    df_test["auto_sarima"]
)

print(f"MAPE of auto-ARIMA: {100*mape_auto_arima:.2f}%")
print(f"MAPE of auto-SARIMA: {100*mape_auto_sarima:.2f}%")

### There's more

우리는 **pmdarima** 라이브러리의 auto-ARIMA 프레임워크를 사용하여 더욱 복잡한 모델이나 전체 파이프라인을 추정할 수 있습니다. 여기에는 타겟 변수를 변환하거나 새로운 특징을 추가하는 작업이 포함됩니다. 이 섹션에서는 그 방법을 설명합니다.

1. Import the libraries:

In [ ]:
from pmdarima.pipeline import Pipeline
from pmdarima.preprocessing import FourierFeaturizer
from pmdarima.preprocessing import LogEndogTransformer
from pmdarima import arima

2. Create new features (month dummies) and split them into the train/test sets:

첫 번째 모델에서는 추가적인 특징(외생 변수)을 포함하여 ARIMA 모델을 훈련합니다. 실험적으로, 각 관측치가 어느 달에 해당하는지를 나타내는 특징을 제공해 봅니다. 만약 이 방법이 효과적이라면, 연간 계절성을 포착하기 위해 SARIMA 모델을 추정할 필요가 없을 수도 있습니다.
우리는 **pd.get_dummies** 함수를 사용하여 더미 변수를 생성합니다. 각 열에는 주어진 관측치가 해당 월에서 나온 것인지를 나타내는 부울 플래그(Boolean flag)가 포함됩니다.
또한, 더미 변수 함정(완전 다중공선성)을 피하기 위해 새 DataFrame에서 첫 번째 열을 제거해야 합니다. 우리는 훈련 세트와 테스트 세트 모두에 새로운 변수를 추가했습니다.

In [ ]:
month_dummies = pd.get_dummies(
    df.index.month, 
    prefix="month_", 
    drop_first=True
)
month_dummies.index = df.index
df = df.join(month_dummies)

df_train = df.iloc[:-TEST_LENGTH]
df_test = df.iloc[-TEST_LENGTH:]

3. Find the best hyperparameters of the ARIMAX model:

그 다음으로, 우리는 **auto_arima** 함수를 사용하여 가장 적합한 모델을 찾습니다. 이 레시피의 3단계와 비교했을 때 달라진 유일한 점은 **exogenous** 인수를 사용하여 외생 변수를 지정해야 했다는 것입니다. 우리는 타겟이 포함된 열을 제외한 모든 열을 지정했습니다. 또는, 추가 변수를 타겟과 동일한 인덱스를 가진 별도의 객체로 유지할 수도 있었습니다.

In [ ]:
auto_arimax = pm.auto_arima(
    df_train[["unemp_rate"]],
    exogenous=df_train.drop(columns=["unemp_rate"]),
    test="adf",
    seasonal=False,
    with_intercept=False,
    stepwise=True,
    suppress_warnings=True,
    trace=True
)
                            
auto_arimax.summary()

Figure 6.33: The summary of the ARIMA model with exogenous variables

우리는 또한 **plot_diagnostics** 메서드를 사용하여 잔차 플롯을 확인했습니다. 연간 계절성과 관련된 자기상관 문제는 더미 변수를 포함함으로써 해결된 것처럼 보입니다.

In [ ]:
auto_arimax.plot_diagnostics(figsize=(18, 14), lags=25);

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_6_32", dpi=200)

Figure 6.34: The diagnostic plots of the ARIMA model with exogenous variables

4. Find the best hyperparameters of the ARIMA pipeline:

마지막으로, 우리는 전체 데이터 변환 및 모델링 파이프라인을 만드는 방법을 보여줍니다. 이 파이프라인은 가장 적합한 ARIMA 모델도 찾습니다. 우리의 파이프라인은 세 가지 단계로 구성됩니다:
- 타겟에 로그 변환을 적용합니다.
- **FourierFeaturizer**를 사용하여 새로운 특징을 생성합니다. (푸리에 급수에 대한 설명은 이 책의 범위를 벗어납니다.) 실제로 이 방법을 사용하면 계절적 모델을 사용하지 않고도 계절성을 고려할 수 있습니다. 이를 통해 조금 더 맥락을 제공하자면, 이는 우리가 월별 더미 변수로 수행한 것과 유사합니다. **FourierFeaturizer** 클래스는 외생 변수 배열로 분해된 계절 푸리에 항을 제공합니다. 우리는 계절 주기 **m**을 지정해야 했습니다.
- **auto-ARIMA** 절차를 사용하여 가장 적합한 모델을 찾습니다. 파이프라인을 사용할 때는 **pm.auto_arima** 함수 대신 **AutoARIMA** 클래스를 사용해야 함을 유념해 주세요. 두 기능 모두 동일한 기능을 제공하지만, 이번에는 파이프라인 기능과 호환되도록 클래스를 사용해야 했습니다.

In [ ]:
auto_arima_pipe = Pipeline([
    ("log_transform", LogEndogTransformer()),
    ("fourier", FourierFeaturizer(m=12)),
    ("arima", arima.AutoARIMA(stepwise=True, trace=1, 
                              error_action="warn",
                              test="adf", seasonal=False, 
                              with_intercept=False, 
                              suppress_warnings=True))
])

auto_arima_pipe.fit(df_train[["unemp_rate"]])

파이프라인을 적용한 결과 로그에 나타난 최적 모델은 다음과 같습니다:

```
Best model:  ARIMA(4,1,0)(0,0,0)[0] intercept
```

파이프라인을 사용하는 가장 큰 장점은 우리가 모든 단계를 직접 수행하지 않아도 된다는 점입니다. 파이프라인을 정의한 다음 시계열 데이터를 입력하여 **fit** 메서드를 호출하면 됩니다. 일반적으로 파이프라인은 (예를 들어 **scikit-learn**에서 제공하는 것처럼) 코드 재사용성을 높이고, 데이터 처리 순서를 명확히 정의하며, 특징 생성 및 데이터 분할 시 잠재적인 데이터 누출을 방지하는 유용한 기능을 제공합니다.

파이프라인 사용의 잠재적인 단점은 일부 작업을 더 이상 쉽게 추적할 수 없다는 점입니다 (중간 결과가 별도의 객체로 저장되지 않음). 따라서 파이프라인의 특정 요소에 접근하는 것이 약간 더 어려워질 수 있습니다. 예를 들어, **auto_arima_pipe.summary()**를 실행하여 적합된 ARIMA 모델의 요약을 얻을 수 없습니다.

---

아래에서는 **predict** 메서드를 사용하여 예측을 생성합니다. 이 과정에서 주목할 만한 사항은 다음과 같습니다:
- 우리는 타겟만 포함된 새로운 DataFrame을 생성했습니다. 이는 이 레시피에서 이전에 생성한 추가 열을 제거하기 위해서입니다.
- 적합된 ARIMAX 모델에서 **predict** 메서드를 사용할 때, 예측을 위해 필요한 외생 변수를 **X** 인수로 전달해야 합니다.
- **predict** 메서드를 사용하여 타겟 변수를 변환하는 파이프라인의 예측 값은 원래 입력과 동일한 스케일로 표현됩니다. 우리 경우, 먼저 원래 시계열 데이터에 로그 변환을 적용한 후 새로운 특징을 추가했습니다. 그런 다음, 모델에서 예측을 얻었으며(여전히 로그 변환된 상태), 마지막으로 지수를 사용하여 예측 값을 원래 스케일로 변환했습니다.

5. Calculate the forecasts and plot them:

In [ ]:
# specify return_conf_int when calling `predict` to get the confidence intervals
results_df = df_test[["unemp_rate"]].copy()
results_df["auto_arimax"] = auto_arimax.predict(
    TEST_LENGTH, 
    X=df_test.drop(columns=["unemp_rate"])
)
results_df["auto_arima_pipe"] = auto_arima_pipe.predict(TEST_LENGTH)
results_df.plot(title="Forecasts of the ARIMAX/pipe models");

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_6_33", dpi=200)

7. Calculate the MAPEs:

In [ ]:
mape_auto_arimax = mean_absolute_percentage_error(results_df["unemp_rate"], 
                                                  results_df["auto_arimax"])

mape_auto_pipe = mean_absolute_percentage_error(results_df["unemp_rate"], 
                                                results_df["auto_arima_pipe"])

print(f"MAPE of auto-ARIMAX: {100*mape_auto_arimax:.2f}%")
print(f"MAPE of auto-pipe: {100*mape_auto_pipe:.2f}%")

이 챕터에서 시도한 모든 ARIMA 모델 중에서 파이프라인 모델이 가장 우수한 성능을 보였습니다. 하지만 여전히 지수 평활법(exponential smoothing) 방법보다 성능이 떨어집니다.

---

팁: **pmdarima** 라이브러리의 ARIMA 모델/파이프라인에서 **predict** 메서드를 사용할 때 **return_conf_int** 인수를 **True**로 설정할 수 있습니다. 이렇게 하면, 메서드가 점 예측뿐만 아니라 해당 신뢰 구간도 반환합니다.

---

### See also

- Hyndman, R. J. & Athanasopoulos, G. 2021. "Fable의 ARIMA 모델링." *Forecasting: Principles and Practice*, 3rd edition, OTexts: Melbourne, Australia. OTexts.com/fpp3. 2022년 5월 8일에 접근 – [https://otexts.com/fpp3/arima-r.html](https://otexts.com/fpp3/arima-r.html).
- Hyndman, R. J. & Khandakar, Y., 2008. "R을 위한 forecast 패키지: 자동 시계열 예측," *Journal of Statistical Software*, 27: 1-22.

## Summary

이 장에서는 시계열 분석과 예측을 위한 전통적인 (통계적) 접근 방식을 다루었습니다. 우리는 시계열 데이터를 추세, 계절성, 그리고 나머지 구성 요소로 분해하는 방법을 배웠습니다. 이 과정은 시계열 데이터를 더 잘 이해하는 데 매우 유용할 수 있지만, 이를 직접적으로 모델링 목적으로도 사용할 수 있습니다.

그 다음으로, 시계열 데이터가 정상성(stationary)을 만족하는지 테스트하는 방법을 설명했습니다. 일부 통계 모델(예: ARIMA)은 정상성을 요구하며, 정상성이 아닌 시계열 데이터를 정상 시계열로 변환하는 방법도 설명했습니다.

마지막으로, 두 가지 가장 인기 있는 통계적 시계열 예측 방법(지수 평활법과 ARIMA 모델)을 탐구했습니다. 우리는 자동 튜닝과 하이퍼파라미터 선택을 포함하는 더 현대적인 방법도 간략하게 다루었습니다.

다음 장에서는 머신러닝 기반의 시계열 예측 방법을 탐구할 것입니다.